<a href="https://colab.research.google.com/github/SruthiM-10/Irrigation-Scheduling-Recommendations-And-Dataset/blob/main/data_processing/(3)_categorization_and_unit_conversions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from openai import OpenAI
import re
import json
import requests
import os

In [3]:
! pip install -q langchain_community langchain_openai faiss-cpu langchain langchain_core langgraph pinecone PyPDF2 doi2pdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

In [5]:
! pip install pydantic

In [7]:
import time
import uuid
import requests
from langchain_community.document_loaders import WebBaseLoader
from typing import Dict, List, Any, Tuple, Optional, Union
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
import pickle
from pydantic import v1 as pydantic_v1
from pydantic import Field, conlist, BaseModel

import warnings

warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    module='langchain_openai.chat_models.base'
)

In [8]:
# Set up OpenAI API
os.environ["OPENAI_API_KEY"] = ""
# mg's acccount

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
def setup_llm() -> Optional[ChatOpenAI]:
    if not os.environ["OPENAI_API_KEY"]:
        print("ERROR: OPENAI_API_KEY is not set. Please set your OpenAI API key.")
        return None

    try:
        # Set up the OpenAI model
        return ChatOpenAI(model="gpt-4o-mini", temperature=0)
    except Exception as e:
        print(f"Error setting up LLM: {e}")
        return None

def call_gpt_api(llm_type, prompt: str, max_tokens: int = 1000) -> str:
    """Call GPT API with error handling"""
    try:
        structured_llm = llm.with_structured_output(llm_type)
        system_message_prompt = [SystemMessage(content=prompt)]
        response = structured_llm.invoke(system_message_prompt)
        return response
    except Exception as e:
        print(f"Error calling GPT API: {e}")
        return ""

llm = setup_llm()

In [10]:
samples = pd.read_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files (11 16 25)/Extra analysis/cleaned_samples (6)_filled.csv")
samples = samples.drop("Unnamed: 0", axis= 1)
samples

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,latitude_decimal_degrees,latitude_notes,lat_filled,longitude,...,water_productivity_kg_per_m^3,water_productivity_notes,water_productivity_valid,water_productivity_filled,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,116.4442° E,...,17.370000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,116.4442° E,...,20.520000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,116.4442° E,...,25.070000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,8.400000,Converted from degrees and minutes format (8°2...,False,39°21′E,...,13.583452,(70% ETc),False,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes format (23º...,False,51º57′W,...,13.583452,NaN,NaN,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,"100% ETc, 4-day interval","136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,34.333333,Original value was in degrees and minutes form...,False,108°04′E,...,19.300000,Autumn 2023,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10
310,Timing of last irrigation,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,14.401180,NaN,True,NaN,...,0.710000,Maximum value of the range 0.45-0.71 kg/m3,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8
311,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,False,86°56′E,...,2.600000,(W 0.8),True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9
312,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original val

# Irrigation Scheduling Method

In [ ]:
print(len(samples["irrigation_scheduling_method"].unique()))
samples["irrigation_scheduling_method"].unique()

114


array(['Soil moisture sensor-based method (T-VWC)',
       'Accumulated radiation combined with soil moisture sensor-based method (Rn-VWC)',
       'Crop evapotranspiration estimated method using the hourly PM-ETo equation with an improved calculation of Kc (T-ETc)',
       'Soil water balance method', 'Regulated Deficit Irrigation',
       'Furrow irrigation', 'Manageable Allowed Depletion (MAD) 60%',
       'Manageable Allowed Depletion (MAD) 80%',
       'Manageable Allowed Depletion (MAD) 100%',
       'Manageable Allowed Depletion (MAD) 120%',
       'Manageable Allowed Depletion (MAD) 140%',
       'Full irrigation (control)', 'Deficit irrigation 30% (S1)',
       'Deficit irrigation 30% (S2)', 'Deficit irrigation 30% (S3)',
       'Deficit irrigation 30% (S4)', 'Deficit irrigation 50% (S1)',
       'Deficit irrigation 50% (S2)', 'Deficit irrigation 50% (S3)',
       'Deficit irrigation 50% (S4)', 'Excess irrigation (150% of ETc)',
       'Fertigation', 'I1=irrigation on both sid

In [ ]:
def categorize_method(method):
    method_lower = method.lower()

    # Soil moisture-based
    if any(k in method_lower for k in ['soil moisture', 'soil water', 'wetting front', 'rtsm']):
        return 'Soil Moisture-Based'

    # Evapotranspiration-based
    elif any(k in method_lower for k in ['evapotranspiration', 'penman', 'blaney', 'pan evaporation', 'et0', 'iw/cpe', 'eto', 'crop water requirement', '100% etc, 4-day interval']):
        return 'Evapotranspiration-Based'

    # Deficit / Partial irrigation
    elif any(k in method_lower for k in [
        'deficit', 'partial', 'mad', 'rdi', 'prz', 'prid',
        'manageable allowed depletion', 'maximum allowable depletion',
        'excess irrigation', 'i1=', 'i2=', 'i 1', 'i 2', 'i 3', 'i 4', 'i 5',
        'non-stressed', 'post-anthesis water stress', 'irrigation after',
        'asmdl'  # catches 60%-140% ASMDL
    ]):
        return 'Deficit/Partial Irrigation'

    # Conventional / Fixed scheduling
    elif any(k in method_lower for k in [
        'furrow', 'conventional', 'control', 'fixed', 'producer',
        'traditional', 'full irrigation', 'drip', 'film-mulched', 'pre-sowing',
        'rainfed', 'once every', 'trickle point', 'trickle line', 'timing of last irrigation','fertigation'
    ]):
        return 'Conventional/Fixed Scheduling'

    # Plant- or climate-based
    elif any(k in method_lower for k in ['canopy', 'radiation', 'climate', 'plant-based', 'temperature', 'ism']):
        return 'Plant or Climate-Based'

    # Anything else
    else:
        return 'Other'

samples_clean = samples.copy()
samples_clean['irrigation_scheduling_method_standardized'] = samples_clean['irrigation_scheduling_method'].apply(categorize_method)
samples_clean

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,latitude_decimal_degrees,latitude_notes,lat_filled,longitude,...,water_productivity_notes,water_productivity_valid,water_productivity_filled,file_path,doi,doi_url,processed_at,content source,relevance_score,irrigation_scheduling_method_standardized
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,116.4442° E,...,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,Soil Moisture-Based
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,116.4442° E,...,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,Soil Moisture-Based
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,116.4442° E,...,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,Evapotranspiration-Based
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,8.400000,Converted from degrees and minutes format (8°2...,False,39°21′E,...,(70% ETc),False,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9,Soil Moisture-Based
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes format (23º...,False,51º57′W,...,NaN,NaN,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9,Deficit/Partial Irrigation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,"100% ETc, 4-day interval","136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,34.333333,Original value was in degrees and minutes form...,False,108°04′E,...,Autumn 2023,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10,Evapotranspiration-Based
310,Timing of last irrigation,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,14.401180,NaN,True,NaN,...,Maximum value of the range 0.45-0.71 kg/m3,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8,Conventional/Fixed Scheduling
311,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,False,86°56′E,...,(W 0.8),True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9,Conventional/Fixed Scheduling
312,Film-Mulched Drip Irrigation,"6218.07 kg ha−

In [ ]:
samples_clean["irrigation_scheduling_method_standardized"].unique()

array(['Soil Moisture-Based', 'Evapotranspiration-Based',
       'Deficit/Partial Irrigation', 'Conventional/Fixed Scheduling',
       'Plant or Climate-Based'], dtype=object)

In [ ]:
categories = ["Soil Moisture-Based", "Evapotranspiration-Based", "Deficit/Partial Irrigation", "Conventional/Fixed Scheduling", "Plant or Climate-Based", "Other"]
info = []
for category in categories:
  info.append({
      'category': category,
      'total_samples': len(samples_clean),
      'samples': len(samples_clean.loc[samples_clean['irrigation_scheduling_method_standardized'] == category]),
      'percentage': float(len(samples_clean.loc[samples_clean['irrigation_scheduling_method_standardized'] == category])/len(samples_clean) * 100.0)
  })
pd.DataFrame(info)

,category,total_samples,samples,percentage
0,Soil Moisture-Based,314,28,8.917197
1,Evapotranspiration-Based,314,35,11.146497
2,Deficit/Partial Irrigation,314,89,28.343949
3,Conventional/Fixed Scheduling,314,158,50.318471
4,Plant or Climate-Based,314,4,1.273885
5,Other,314,0,0.000000


In [ ]:
combined_samples = samples_clean[["irrigation_scheduling_method", "irrigation_scheduling_method_standardized", "plant_yield", "plant_yield_kg_per_hectare", "plant_yield_notes",
                  "location", "latitude", "latitude_decimal_degrees", "latitude_notes", "lat_filled", "longitude", "longitude_decimal_degrees", "longitude_notes", "long_filled",
                  "altitude", "altitude_numeric", "altitude_notes", "altitude_filled",
                  "plant_type", "plant_type_standardized", "plant_type_notes",
                  "water_productivity", "water_productivity_kg_per_m^3", "water_productivity_notes", "water_productivity_valid", "water_productivity_filled",
                  "file_path", "doi", "doi_url", "processed_at", "content source", "relevance_score"]]
combined_samples

,irrigation_scheduling_method,irrigation_scheduling_method_standardized,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,latitude_decimal_degrees,latitude_notes,lat_filled,...,water_productivity_kg_per_m^3,water_productivity_notes,water_productivity_valid,water_productivity_filled,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),Soil Moisture-Based,66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,17.370000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,Soil Moisture-Based,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,20.520000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,Evapotranspiration-Based,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,25.070000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,Soil Moisture-Based,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,8.400000,Converted from degrees and minutes format (8°2...,False,...,13.583452,(70% ETc),False,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,Deficit/Partial Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes format (23º...,False,...,13.583452,NaN,NaN,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,"100% ETc, 4-day interval",Evapotranspiration-Based,"136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,34.333333,Original value was in degrees and minutes form...,False,...,19.300000,Autumn 2023,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10
310,Timing of last irrigation,Conventional/Fixed Scheduling,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,14.401180,NaN,True,...,0.710000,Maximum value of the range 0.45-0.71 kg/m3,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8
311,Film-Mulched Drip Irrigation,Conventional/Fixed Scheduling,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,False,...,2.600000,(W 0.8),True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9
312,Film-Mulched Drip I

In [ ]:
combined_samples.to_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (8).csv", index= False)

# Plant Yield

In [ ]:
print(len(samples["plant_yield"].unique()))
samples["plant_yield"].unique()

196


array(['66.77 t ha−1', '58.98 t ha−1', '64.54 t ha−1',
       '5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% ETc), 4,555 kg ha−1 (70% ETc), 3,475 kg ha−1 (55% ETc), 2,510 kg ha−1 (40% ETc)',
       '2.406 kg m-2', '2.540 kg m-2', '2.701 kg m-2', '3.06 kg m-2',
       '6.1 t ha-1', '6.4 t ha-1', '0.2 t ha-1', '3.7 t ha-1',
       '3.5 t ha-1', '2.2 t ha-1', '9.9 t ha-1', '5.1 t ha-1',
       '1.4 t ha-1', '164.9 t ha-1', '752.3 t ha-1', '5.5 t ha-1',
       '6.0 t ha-1', '3.6 t ha-1', '58.6 t ha-1', '26.1 t ha-1',
       '2.3 t ha-1', '2.6 t ha-1', '34.7 t ha-1', '33.8 t ha-1',
       '18.15 qt/ha', '24.84 qt/ha', '24.51 qt/ha', '21.99 qt/ha',
       '17.36 qt/ha', '3.060 kg m-2', '3384.3 kg/ha', '2626.0 kg/ha',
       '3042.8 kg/ha', '2736.5 kg/ha', '3087.4 kg/ha', '2388.0 kg/ha',
       '2271.3 kg/ha', '2228.0 kg/ha', '2968.7 kg/ha', '4037.2 kg/ha',
       '9731.43 kg ha-1', '5455 kg/ha', '5228 kg/ha',
       '5.150 t ha−1 (N-S), 3.894 t ha−1 (S), 2.950 t ha−1 (HS)',
       '4.158 t ha

In [ ]:
class PlantYieldItem(BaseModel):
  current_unit: str
  conversion_factor: Optional[float] = None
  original_value: float
  cleaning_notes: str

class PlantYieldLLM(BaseModel):
    """List of references extracted from a paper."""
    yield_list: conlist(PlantYieldItem, min_items=0) = Field(
        description="List of information for plant yield values"
    )

def standardize_plant_yield(df: pd.DataFrame) -> pd.DataFrame:
    """Convert plant_yield to standardized kg/hectare"""
    print("Standardizing plant_yield...")

    # Get sample values to understand the format
    sample_yields = df['plant_yield'].dropna().head(10).tolist()
    print(f"Sample yield values: {sample_yields}")

    prompt = """
You are given plant yield value(s) from agricultural data. Your task is to determine the current unit of these values and how to convert their units to kg/hectare.
If there are multiple comma-separated values, convert and return them all as separate objects.

Analyze the format of each value and provide:
  1. current_unit: The name of the unit this value appears to be in.
  2. conversion_factor: a float conversion factor from this unit to kg/hectare.
     If this current unit is per plant or pot or if isn't per area at all, return null for the conversion factor.
     Here are some common conversion factors -
        1 ton = 907.185 kg
        1 hectare = 10000 m^2
        1 hectare = 2.381 fed
        1 hectare = 107639 ft^2
        1 ton/fed = 453.57 kg/ha
        1 qt = 0.9464 kg

  3. original_value: the original float value in the original unit.
  4. cleaning_notes: notes about any extra information associated with each value in the input string

Return ONLY as a JSON array of objects.

Yield values: {values}
    """

    extra_rows_to_add = []

    # Extract numerical values and convert
    def extract_and_convert_yield(row):
        value = row["plant_yield"]
        if pd.isna(value):
            return np.nan

        # Extract numbers from string
        response = call_gpt_api(llm_type= PlantYieldLLM, prompt= prompt.format(values=value)).yield_list
        analysis = response[0].dict()
        print("Original value: {}".format(value))
        print(f"First base val {analysis['original_value']}")
        print(analysis)
        if len(response) > 1:
          print("Other values")
          for i in range(1, len(response)):
            val = response[i]
            other_analysis = val.dict()
            print(other_analysis)
            new_row = row.copy()
            if other_analysis["conversion_factor"] is not None:
              # removing rows with unclear units for plant yield
              new_row["plant_yield_kg_per_hectare"] = other_analysis["original_value"] * other_analysis["conversion_factor"]
            new_row['plant_yield_notes'] = other_analysis['cleaning_notes']
            extra_rows_to_add.append(new_row)

        if analysis["conversion_factor"] is None:
          row['plant_yield_kg_per_hectare'] = np.nan;
        else:
          row['plant_yield_kg_per_hectare'] = analysis["original_value"] * analysis['conversion_factor']
        row['plant_yield_notes'] = analysis['cleaning_notes']

        return row

    df = df.apply(extract_and_convert_yield, axis= 1)

    return df, extra_rows_to_add

In [ ]:
df, extra_rows_to_add = standardize_plant_yield(samples)
df

Standardizing plant_yield...
Sample yield values: ['66.77 t ha−1', '58.98 t ha−1', '64.54 t ha−1', '5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% ETc), 4,555 kg ha−1 (70% ETc), 3,475 kg ha−1 (55% ETc), 2,510 kg ha−1 (40% ETc)', '2.406 kg m-2', '2.540 kg m-2', '2.701 kg m-2', '3.06 kg m-2', '5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% ETc), 4,555 kg ha−1 (70% ETc), 3,475 kg ha−1 (55% ETc), 2,510 kg ha−1 (40% ETc)', '6.1 t ha-1']


/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1915: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


Original value: 66.77 t ha−1
First base val 66.77
{'current_unit': 'ton/ha', 'conversion_factor': 907.185, 'original_value': 66.77, 'cleaning_notes': 'This value is given in tons per hectare.'}


/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1915: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


Original value: 58.98 t ha−1
First base val 58.98
{'current_unit': 'ton/ha', 'conversion_factor': 907.185, 'original_value': 58.98, 'cleaning_notes': 'Yield value is given in tons per hectare.'}


...

/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1915: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


Original value: 1669-2405 kg/ha
First base val 1669.0
{'current_unit': 'kg/ha', 'conversion_factor': 1.0, 'original_value': 1669.0, 'cleaning_notes': 'Value is already in kg/ha.'}
Other values
{'current_unit': 'kg/ha', 'conversion_factor': 1.0, 'original_value': 2405.0, 'cleaning_notes': 'Value is already in kg/ha.'}


/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1915: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


Original value: 6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)
First base val 6218.07
{'current_unit': 'kg/ha', 'conversion_factor': 1.0, 'original_value': 6218.07, 'cleaning_notes': 'Yield value for the year 2018.'}
Other values
{'current_unit': 'kg/ha', 'conversion_factor': 1.0, 'original_value': 5660.59, 'cleaning_notes': 'Yield value for the year 2019.'}


,irrigation_scheduling_method,plant_yield,location,latitude,longitude,altitude,plant_type,water_productivity,file_path,doi,doi_url,processed_at,content source,relevance_score,plant_yield_kg_per_hectare,plant_yield_notes
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),17.37 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,60572.74245,This value is given in tons per hectare.
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),20.52 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,53505.77130,Yield value is given in tons per hectare.
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),25.07 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,58549.71990,Value is given in tons per hectare.
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...","Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),1.42 kg m−3 (70% ETc),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9,5085.00000,100% ETc
4,Regulated Deficit Irrigation,2.406 kg m-2,"Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9,24060.00000,"Value is in kilograms per square meter, which ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,Non-irrigated,"12,108 kg/ha for corn, 3,569 kg/ha for soybean",Dry corner of each site,NaN,NaN,NaN,"Corn, Soybean",NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9,12108.00000,Yield value for corn.
242,Non-stressed (I1),"4,974 kg/ha (2006-2007), 4,802 kg/ha (2007-2008)","Shiraz Agricultural Research Station, Iran",29°50'N,52°46'E,1810 m,Winter wheat,0.66 to 1.22 kg m/C03,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.2166/ws.2021.246,https://doi.org/10.2166/ws.2021.246,2025-09-12 23:48:15,pdf,9,4974.00000,Yield value for the year 2006-2007.
243,Post-anthesis water stress (I2),"4,272 kg/ha (2006-2007), 4,485 kg/ha (2007-2008)","Shiraz Agricultural Research Station, Iran",29°50'N,52°46'E,1810 m,Winter wheat,0.63 to 1.08 kg m/C03,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.2166/ws.2021.246,https://doi.org/10.2166/ws.2021.246,2025-09-12 23:48:15,pdf,9,4272.00000,Yield value for the year 2006-2007.
244,Timing of last irrigation,1669-2405 kg/ha,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,NaN,NaN,Cotton (Gossypium hirsutum L.),0.45-0.71 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8,1669.00000,Value is already in kg/ha.


In [ ]:
new_df = pd.concat([df, pd.DataFrame(extra_rows_to_add)], ignore_index= True)
new_df = new_df.dropna(subset=["plant_yield_kg_per_hectare"])
new_df

,irrigation_scheduling_method,plant_yield,location,latitude,longitude,altitude,plant_type,water_productivity,file_path,doi,doi_url,processed_at,content source,relevance_score,plant_yield_kg_per_hectare,plant_yield_notes
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),17.37 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,60572.74245,This value is given in tons per hectare.
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),20.52 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,53505.77130,Yield value is given in tons per hectare.
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),25.07 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,58549.71990,Value is given in tons per hectare.
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...","Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),1.42 kg m−3 (70% ETc),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9,5085.00000,100% ETc
4,Regulated Deficit Irrigation,2.406 kg m-2,"Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9,24060.00000,"Value is in kilograms per square meter, which ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,Non-irrigated,"12,108 kg/ha for corn, 3,569 kg/ha for soybean",Dry corner of each site,NaN,NaN,NaN,"Corn, Soybean",NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9,3569.00000,Yield value for soybean.
305,Non-stressed (I1),"4,974 kg/ha (2006-2007), 4,802 kg/ha (2007-2008)","Shiraz Agricultural Research Station, Iran",29°50'N,52°46'E,1810 m,Winter wheat,0.66 to 1.22 kg m/C03,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.2166/ws.2021.246,https://doi.org/10.2166/ws.2021.246,2025-09-12 23:48:15,pdf,9,4802.00000,Yield value for the year 2007-2008.
306,Post-anthesis water stress (I2),"4,272 kg/ha (2006-2007), 4,485 kg/ha (2007-2008)","Shiraz Agricultural Research Station, Iran",29°50'N,52°46'E,1810 m,Winter wheat,0.63 to 1.08 kg m/C03,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.2166/ws.2021.246,https://doi.org/10.2166/ws.2021.246,2025-09-12 23:48:15,pdf,9,4485.00000,Yield value for the year 2007-2008.
307,Timing of last irrigation,1669-2405 kg/ha,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,NaN,NaN,Cotton (Gossypium hirsutum L.),0.45-0.71 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8,2405.00000,Value is already in kg/ha.


In [ ]:
new_df['plant_yield'].unique()

array(['66.77 t ha−1', '58.98 t ha−1', '64.54 t ha−1',
       '5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% ETc), 4,555 kg ha−1 (70% ETc), 3,475 kg ha−1 (55% ETc), 2,510 kg ha−1 (40% ETc)',
       '2.406 kg m-2', '2.540 kg m-2', '2.701 kg m-2', '3.06 kg m-2',
       '6.1 t ha-1', '6.4 t ha-1', '0.2 t ha-1', '3.7 t ha-1',
       '3.5 t ha-1', '2.2 t ha-1', '9.9 t ha-1', '5.1 t ha-1',
       '1.4 t ha-1', '164.9 t ha-1', '752.3 t ha-1', '5.5 t ha-1',
       '6.0 t ha-1', '3.6 t ha-1', '58.6 t ha-1', '26.1 t ha-1',
       '2.3 t ha-1', '2.6 t ha-1', '34.7 t ha-1', '33.8 t ha-1',
       '18.15 qt/ha', '24.84 qt/ha', '24.51 qt/ha', '21.99 qt/ha',
       '17.36 qt/ha', '3.060 kg m-2', '3384.3 kg/ha', '2626.0 kg/ha',
       '3042.8 kg/ha', '2736.5 kg/ha', '3087.4 kg/ha', '2388.0 kg/ha',
       '2271.3 kg/ha', '2228.0 kg/ha', '2968.7 kg/ha', '4037.2 kg/ha',
       '9731.43 kg ha-1', '5455 kg/ha', '5228 kg/ha',
       '5.150 t ha−1 (N-S), 3.894 t ha−1 (S), 2.950 t ha−1 (HS)',
       '4.158 t ha

In [ ]:
new_df.to_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (2).csv");

# Plant type

In [ ]:
samples = pd.read_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (2).csv")
samples.drop("Unnamed: 0", axis= 1, inplace= True)
samples

,irrigation_scheduling_method,plant_yield,location,latitude,longitude,altitude,plant_type,water_productivity,file_path,doi,doi_url,processed_at,content source,relevance_score,plant_yield_kg_per_hectare,plant_yield_notes
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),17.37 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,60572.74245,This value is given in tons per hectare.
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),20.52 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,53505.77130,Yield value is given in tons per hectare.
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),25.07 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,58549.71990,Value is given in tons per hectare.
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...","Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),1.42 kg m−3 (70% ETc),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9,5085.00000,100% ETc
4,Regulated Deficit Irrigation,2.406 kg m-2,"Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9,24060.00000,"Value is in kilograms per square meter, which ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,Non-irrigated,"12,108 kg/ha for corn, 3,569 kg/ha for soybean",Dry corner of each site,NaN,NaN,NaN,"Corn, Soybean",NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9,3569.00000,Yield value for soybean.
275,Non-stressed (I1),"4,974 kg/ha (2006-2007), 4,802 kg/ha (2007-2008)","Shiraz Agricultural Research Station, Iran",29°50'N,52°46'E,1810 m,Winter wheat,0.66 to 1.22 kg m/C03,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.2166/ws.2021.246,https://doi.org/10.2166/ws.2021.246,2025-09-12 23:48:15,pdf,9,4802.00000,Yield value for the year 2007-2008.
276,Post-anthesis water stress (I2),"4,272 kg/ha (2006-2007), 4,485 kg/ha (2007-2008)","Shiraz Agricultural Research Station, Iran",29°50'N,52°46'E,1810 m,Winter wheat,0.63 to 1.08 kg m/C03,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.2166/ws.2021.246,https://doi.org/10.2166/ws.2021.246,2025-09-12 23:48:15,pdf,9,4485.00000,Yield value for the year 2007-2008.
277,Timing of last irrigation,1669-2405 kg/ha,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,NaN,NaN,Cotton (Gossypium hirsutum L.),0.45-0.71 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8,2405.00000,Value is already in kg/ha.


In [ ]:
samples["plant_yield_notes"].unique()

array(['This value is given in tons per hectare.',
       'Yield value is given in tons per hectare.',
       'Value is given in tons per hectare.', '100% ETc',
       'Value is in kilograms per square meter, which can be converted to kilograms per hectare.',
       'Value is in kg per square meter, which can be converted to kg/hectare.',
       'The value is in tons per hectare.',
       'This value is in tons per hectare.',
       'Value is in tons per hectare.',
       'Yield is given in tons per hectare.',
       'Standard yield measurement in agriculture.',
       'The value is given in quarts per hectare.',
       'Yield is given in quarts per hectare.',
       'Yield value is given in quarts per hectare.',
       'Yield value in quarts per hectare.',
       'Value is in kg per square meter, which can be converted to kg/hectare by multiplying by 10000.',
       'Value is already in kg/ha.', 'Value is already in kg/hectare.',
       'Yield in tons per hectare.',
       'Yield in t

In [ ]:
print(len(samples["plant_type"].unique()))
samples["plant_type"].unique()

In [ ]:
class PlantTypeItem(BaseModel):
  new_type: str
  cleaning_notes: str

class PlantTypeLLM(BaseModel):
    type_list: conlist(PlantTypeItem, min_items=0) = Field(
        description="List of information for plant types"
    )

def standardize_plant_type(df: pd.DataFrame, notes: bool = False) -> pd.DataFrame:
    """Convert plant_type to standardized categories"""
    print("Standardizing plant_type...")

    # Get sample values to understand the format
    sample_types = df['plant_type'].dropna().head(10).tolist()
    print(f"Sample plant type values: {sample_types}")

    notes_prompt = """
You are given plant type(s) from agricultural data. Your task is to classify the input using one of the following plant types.

Here are the valid categories that can be returned:
Bell pepper
Guava
Cucumber
Wheat
Broccoli
Maize
Rice
Sorghum
Millet
Barley
Soybean
Potato
Sweet potato
Cotton
Sugarcane
Sugar beet
Groundnut
Rapeseed
Sunflower
Tomato
Apple
Tea
Tobacco
Cabbage
Papaya
Grapes
Haricot bean
Green Pea
Mustard
Date Palm

Rules:
- Scientific names of plants must be associated to the current category.
- Maize is the same as corn, and bell peppers are the same as capsicum

Task:
Analyze the below notes and identify if there is a plant type mentioned.

If so, analyze the format of this plant type and provide:
  1. new_type: classify this plant type into one of the above categories
  2. cleaning_notes: return any extra information in input_string that isn't in new_type or "NA" if no extra information can be found

If there is no plant type mentioned in the notes, return "NA" for new_type and cleaning_notes.

Return ONLY as a JSON array of objects.

Notes: {notes}
    """

    prompt = """
You are given plant type(s) from agricultural data. Your task is to classify the input using one of the following plant types.
If there are multiple comma-separated values in the input, classify and return all of the plant types as separate objects.

Here are the valid categories that can be returned:
Bell pepper
Guava
Cucumber
Wheat
Broccoli
Maize
Rice
Sorghum
Millet
Barley
Soybean
Potato
Sweet potato
Cotton
Sugarcane
Sugar beet
Groundnut
Rapeseed
Sunflower
Tomato
Apple
Tea
Tobacco
Cabbage
Papaya
Grapes
Haricot bean
Green Pea
Mustard
Date Palm

Rules:
- Scientific names of plants must be associated to the current category.
- Maize is the same as corn, and bell peppers are the same as capsicum

Task:
Analyze the format of the given plant type and provide:
  1. new_type: classify this plant type into one of the above categories
  2. cleaning_notes: return any extra information in input_string that isn't in new_type or "NA" if no extra information can be found

Return ONLY as a JSON array of objects.

Plant Value(s): {value}
    """

    extra_rows_to_add = []

    # Extract numerical values and convert
    def extract_and_convert_type(row):
        value = row["plant_type"]
        if pd.isna(value):
            return np.nan

        # Extract numbers from string
        if notes:
            print(f"Input String: {row['plant_yield_notes']}")
        else:
            print(f"Input String: {value}")

        if notes:
            response = call_gpt_api(llm_type= PlantTypeLLM, prompt= notes_prompt.format(notes=row["plant_yield_notes"])).type_list
        else:
            response = call_gpt_api(llm_type= PlantTypeLLM, prompt= prompt.format(value=value)).type_list

        analysis = response[0].dict()
        print(analysis)

        # if there are comma-separated values, response > 1, will be extra_rows_to_add
        if len(response) > 1:
          print("Other values")
          for i in range(1, len(response)):
            val = response[i]
            other_analysis = val.dict()
            print(other_analysis)
            new_row = row.copy()
            new_row["plant_type_standardized"] = other_analysis["new_type"]
            new_row['plant_type_notes'] = other_analysis['cleaning_notes']
            extra_rows_to_add.append(new_row)

        row['plant_type_standardized'] = analysis["new_type"]
        row['plant_type_notes'] = analysis['cleaning_notes']

        return row

    df = df.apply(extract_and_convert_type, axis= 1)

    return df, extra_rows_to_add

In [ ]:
samples_w_notes, extra_rows_to_add = standardize_plant_type(samples, notes= True)
samples_w_notes

Standardizing plant_type...
Sample plant type values: ['Cucumber (Zhongnong 26)', 'Cucumber (Zhongnong 26)', 'Cucumber (Zhongnong 26)', 'Wheat (Triticum aestivum L.)', 'Brassica oleracea var. italica', 'Brassica oleracea var. italica', 'Brassica oleracea var. italica', 'Brassica oleracea var. italica', 'Wheat (Triticum aestivum L.)', 'Wheat']
Input String: This value is given in tons per hectare.


/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1900: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


{'new_type': 'NA', 'cleaning_notes': 'NA'}
Input String: Yield value is given in tons per hectare.


/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1900: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


{'new_type': 'NA', 'cleaning_notes': 'NA'}
Input String: Value is given in tons per hectare.


...

/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1900: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


{'new_type': 'NA', 'cleaning_notes': 'NA'}
Input String: Yield value for the year 2019.


/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1900: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


{'new_type': 'NA', 'cleaning_notes': 'NA'}


,irrigation_scheduling_method,plant_yield,location,latitude,longitude,altitude,plant_type,water_productivity,file_path,doi,doi_url,processed_at,content source,relevance_score,plant_yield_kg_per_hectare,plant_yield_notes,plant_type_standardized,plant_type_notes
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),17.37 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,60572.74245,This value is given in tons per hectare.,NA,NA
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),20.52 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,53505.77130,Yield value is given in tons per hectare.,NA,NA
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),25.07 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,58549.71990,Value is given in tons per hectare.,NA,NA
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...","Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),1.42 kg m−3 (70% ETc),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9,5085.00000,100% ETc,NA,NA
4,Regulated Deficit Irrigation,2.406 kg m-2,"Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9,24060.00000,"Value is in kilograms per square meter, which ...",NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,Non-irrigated,"12,108 kg/ha for corn, 3,569 kg/ha for soybean",Dry corner of each site,NaN,NaN,NaN,"Corn, Soybean",NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9,3569.00000,Yield value for soybean.,Soybean,Yield value for soybean.
275,Non-stressed (I1),"4,974 kg/ha (2006-2007), 4,802 kg/ha (2007-2008)","Shiraz Agricultural Research Station, Iran",29°50'N,52°46'E,1810 m,Winter wheat,0.66 to 1.22 kg m/C03,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.2166/ws.2021.246,https://doi.org/10.2166/ws.2021.246,2025-09-12 23:48:15,pdf,9,4802.00000,Yield value for the year 2007-2008.,NA,NA
276,Post-anthesis water stress (I2),"4,272 kg/ha (2006-2007), 4,485 kg/ha (2007-2008)","Shiraz Agricultural Research Station, Iran",29°50'N,52°46'E,1810 m,Winter wheat,0.63 to 1.08 kg m/C03,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.2166/ws.2021.246,https://doi.org/10.2166/ws.2021.246,2025-09-12 23:48:15,pdf,9,4485.00000,Yield value for the year 2007-2008.,NA,NA
277,Timing of last irrigation,1669-2405 kg/ha,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,NaN,NaN,Cotton (Gossypium hirsutum L.),0.45-0.71 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8,2405.00000,Value is already in kg/ha.,NA,NA


In [ ]:
pd.DataFrame(extra_rows_to_add)

""


In [ ]:
samples_w_notes = samples_w_notes.loc[samples_w_notes["plant_type_standardized"] != "NA"]
samples_w_notes

,irrigation_scheduling_method,plant_yield,location,latitude,longitude,altitude,plant_type,water_productivity,file_path,doi,doi_url,processed_at,content source,relevance_score,plant_yield_kg_per_hectare,plant_yield_notes,plant_type_standardized,plant_type_notes
201,IW/CPE ratio 0.8,"2989 kg/ha (Wheat), 1236 kg/ha (Mustard)","Jabalpur, Madhya Pradesh, India",NaN,NaN,NaN,Wheat + Mustard,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,2025-09-12 23:36:58,pdf,8,2989.0,Yield value for Wheat.,Wheat,Yield value for
202,IW/CPE ratio 0.6,"2595 kg/ha (Wheat), 1104 kg/ha (Mustard)","Jabalpur, Madhya Pradesh, India",NaN,NaN,NaN,Wheat + Mustard,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,2025-09-12 23:36:58,pdf,8,2595.0,Yield value for Wheat.,Wheat,Yield value for
215,Non-irrigated,"12,108 kg/ha for corn, 3,569 kg/ha for soybean",Dry corner of each site,NaN,NaN,NaN,"Corn, Soybean",NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9,12108.0,Yield value for corn.,Maize,Yield value for corn.
272,IW/CPE ratio 0.8,"2989 kg/ha (Wheat), 1236 kg/ha (Mustard)","Jabalpur, Madhya Pradesh, India",NaN,NaN,NaN,Wheat + Mustard,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,2025-09-12 23:36:58,pdf,8,1236.0,Yield value for Mustard.,Mustard,Yield value for Mustard.
273,IW/CPE ratio 0.6,"2595 kg/ha (Wheat), 1104 kg/ha (Mustard)","Jabalpur, Madhya Pradesh, India",NaN,NaN,NaN,Wheat + Mustard,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,2025-09-12 23:36:58,pdf,8,1104.0,Yield value for Mustard.,Mustard,Yield value for Mustard.
274,Non-irrigated,"12,108 kg/ha for corn, 3,569 kg/ha for soybean",Dry corner of each site,NaN,NaN,NaN,"Corn, Soybean",NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9,3569.0,Yield value for soybean.,Soybean,Yield value for soybean.


In [ ]:
other_samples, extra_rows_to_add_1 = standardize_plant_type(samples_w_notes.loc[samples_w_notes["plant_type_standardized"] == "NA"])
other_samples

In [ ]:
other_samples.to_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (3).csv")

In [ ]:
samples = pd.read_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (3).csv")
samples.drop("Unnamed: 0", axis=1, inplace=True)
samples = pd.concat([samples, samples_w_notes], ignore_index=True)
samples

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,longitude,altitude,plant_type,plant_type_standardized,plant_type_notes,water_productivity,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),17.37 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),20.52 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),25.07 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),Wheat,(Triticum aestivum L.),1.42 kg m−3 (70% ETc),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,Broccoli,Brassica oleracea var. italica is the scientif...,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,IW/CPE ratio 0.6,"2595 kg/ha (Wheat), 1104 kg/ha (Mustard)",2595.00000,Yield value for Wheat.,"Jabalpur, Madhya Pradesh, India",NaN,NaN,NaN,Wheat + Mustard,Wheat,Yield value for,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,2025-09-12 23:36:58,pdf,8
278,Non-irrigated,"12,108 kg/ha for corn, 3,569 kg/ha for soybean",12108.00000,Yield value for corn.,Dry corner of each site,NaN,NaN,NaN,"Corn, Soybean",Maize,Yield value for corn.,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9
279,IW/CPE ratio 0.8,"2989 kg/ha (Wheat), 1236 kg/ha (Mustard)",1236.00000,Yield value for Mustard.,"Jabalpur, Madhya Pradesh, India",NaN,NaN,NaN,Wheat + Mustard,Mustard,Yield value for Mustard.,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,2025-09-12 23:36:58,pdf,8
280,IW/CPE ratio 0.6,"2595 kg/ha (Wheat), 1104 kg/ha (Mustard)",1104.00000,Yield value for Mustard.,"Jabalpur, Madhya Pradesh, India",NaN,NaN,NaN,Wheat + Mustard,Mustard,Yield value for Mustard.,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,2025-09-12 23:36:58,pdf,8


In [ ]:
df = pd.concat([df, pd.DataFrame(extra_rows_to_add)], ignore_index=True)
df

,irrigation_scheduling_method,plant_yield,location,latitude,longitude,altitude,plant_type,water_productivity,file_path,doi,doi_url,processed_at,content source,relevance_score,plant_yield_kg_per_hectare,plant_yield_notes,plant_type_standardized,plant_type_notes
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),17.37 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,60572.74245,This value is given in tons per hectare.,Cucumber,(Zhongnong 26)
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),20.52 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,53505.77130,Yield value is given in tons per hectare.,Cucumber,(Zhongnong 26)
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),25.07 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,58549.71990,Value is given in tons per hectare.,Cucumber,(Zhongnong 26)
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...","Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),1.42 kg m−3 (70% ETc),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9,5085.00000,100% ETc,Wheat,(Triticum aestivum L.)
4,Regulated Deficit Irrigation,2.406 kg m-2,"Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9,24060.00000,"Value is in kilograms per square meter, which ...",Broccoli,Brassica oleracea var. italica is the scientif...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,Timing of last irrigation,1669-2405 kg/ha,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,NaN,NaN,Cotton (Gossypium hirsutum L.),0.45-0.71 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8,2405.00000,Value is already in kg/ha.,Cotton,(Gossypium hirsutum L.)
272,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)","Korla, Xinjiang, China",40°53′N,86°56′E,900 m,Gossypium hirstum L.,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9,5660.59000,Yield value for the year 2019.,Cotton,Gossypium hirstum L. is a scientific name for ...
273,Maximum Allowable Depletion (MAD) 60%,"14,681 kg/ha","Burlington, Michigan",NaN,NaN,NaN,"Corn, Soybean",NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9,14681.00000,Value is already in kg/ha.,Soybean,NA
274,Maximum Allowable Depletion (MAD) 50%,"15,183 kg/ha","Union City, Michigan",NaN,NaN,NaN,"Corn, Soybean",NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9,15183.00000,Value is already in kg/ha.,Soybean,NA


In [ ]:
df = df[["irrigation_scheduling_method", "plant_yield", "plant_yield_kg_per_hectare", "plant_yield_notes",
                  "location", "latitude", "longitude", "altitude", "plant_type", "plant_type_standardized", "plant_type_notes",
                  "water_productivity", "file_path", "doi", "doi_url", "processed_at", "content source", "relevance_score"]]
df

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,longitude,altitude,plant_type,plant_type_standardized,plant_type_notes,water_productivity,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),17.37 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),20.52 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),25.07 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),Wheat,(Triticum aestivum L.),1.42 kg m−3 (70% ETc),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,Broccoli,Brassica oleracea var. italica is the scientif...,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,Timing of last irrigation,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,NaN,NaN,Cotton (Gossypium hirsutum L.),Cotton,(Gossypium hirsutum L.),0.45-0.71 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8
272,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,86°56′E,900 m,Gossypium hirstum L.,Cotton,Gossypium hirstum L. is a scientific name for ...,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9
273,Maximum Allowable Depletion (MAD) 60%,"14,681 kg/ha",14681.00000,Value is already in kg/ha.,"Burlington, Michigan",NaN,NaN,NaN,"Corn, Soybean",Soybean,NA,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9
274,Maximum Allowable Depletion (MAD) 50%,"15,183 kg/ha",15183.00000,Value is already in kg/ha.,"Union City, Michigan",NaN,NaN,NaN,"Corn, Soybean",Soybean,NA,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9


In [ ]:
df.to_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (3).csv")

## 5 category version

In [11]:
samples = pd.read_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files (11 16 25)/Extra analysis/cleaned_samples (6)_filled.csv")
samples.drop("Unnamed: 0", axis= 1, inplace= True)
samples

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,latitude_decimal_degrees,latitude_notes,lat_filled,longitude,...,water_productivity_kg_per_m^3,water_productivity_notes,water_productivity_valid,water_productivity_filled,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,116.4442° E,...,17.370000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,116.4442° E,...,20.520000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,116.4442° E,...,25.070000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,8.400000,Converted from degrees and minutes format (8°2...,False,39°21′E,...,13.583452,(70% ETc),False,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes format (23º...,False,51º57′W,...,13.583452,NaN,NaN,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,"100% ETc, 4-day interval","136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,34.333333,Original value was in degrees and minutes form...,False,108°04′E,...,19.300000,Autumn 2023,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10
310,Timing of last irrigation,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,14.401180,NaN,True,NaN,...,0.710000,Maximum value of the range 0.45-0.71 kg/m3,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8
311,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,False,86°56′E,...,2.600000,(W 0.8),True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9
312,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original val

In [16]:
samples.columns

Index(['irrigation_scheduling_method', 'plant_yield',
       'plant_yield_kg_per_hectare', 'plant_yield_notes', 'location',
       'latitude', 'latitude_decimal_degrees', 'latitude_notes', 'lat_filled',
       'longitude', 'longitude_decimal_degrees', 'longitude_notes',
       'long_filled', 'altitude', 'altitude_numeric', 'altitude_notes',
       'altitude_filled', 'plant_type', 'plant_type_standardized',
       'plant_type_notes', 'water_productivity',
       'water_productivity_kg_per_m^3', 'water_productivity_notes',
       'water_productivity_valid', 'water_productivity_filled', 'file_path',
       'doi', 'doi_url', 'processed_at', 'content source', 'relevance_score'],
      dtype='object')

In [36]:
info = []
for name, group in samples.groupby("plant_type_standardized"):
  info.append({
      'type': name,
      'sample_number': len(group),
      'Yield_avg': group["plant_yield_kg_per_hectare"].sum() / len(group),
      'Yield_range': group["plant_yield_kg_per_hectare"].max() - group["plant_yield_kg_per_hectare"].min(),
      'Lat_avg': group["latitude_decimal_degrees"].sum() / len(group),
      'Lat_range': group["latitude_decimal_degrees"].max() - group["latitude_decimal_degrees"].min(),
      'Long_avg': group["longitude_decimal_degrees"].sum() / len(group),
      'Long_rang': group["longitude_decimal_degrees"].max() - group["longitude_decimal_degrees"].min(),
      'Altitude_avg': group["altitude_numeric"].sum() / len(group),
      'Altitude_range': group["altitude_numeric"].max() - group["altitude_numeric"].min(),
      'Water_productivity_avg': group["water_productivity_kg_per_m^3"].sum() / len(group),
      'Water_productivity_range': group["water_productivity_kg_per_m^3"].max() - group["water_productivity_kg_per_m^3"].min()
  })
info = pd.DataFrame(info)
info["sum"] = info.drop('type', axis= 1).sum(axis = 1)

In [38]:
info.sort_values(by= ["sum"])

,type,sample_number,Yield_avg,Yield_range,Lat_avg,Lat_range,Long_avg,Long_rang,Altitude_avg,Altitude_range,Water_productivity_avg,Water_productivity_range,sum
13,Mustard,2,1170.000000,132.000000,23.170152,0.000000,79.932451,0.000000,400.000000,0.0,13.583452,0.000000,1820.686055
17,Rice,2,181.437000,0.000000,35.000074,0.000000,104.999927,0.000000,2001.000000,0.0,3.927211,0.000000,2328.364212
24,Tea,2,2086.525500,0.000000,35.000074,0.000000,104.999927,0.000000,2001.000000,0.0,0.512824,0.000000,4230.038325
25,Tobacco,2,2358.681000,0.000000,35.000074,0.000000,104.999927,0.000000,2001.000000,0.0,1.522124,0.000000,4503.203125
12,Millet,2,3175.147500,0.000000,35.000074,0.000000,104.999927,0.000000,2001.000000,0.0,1.261733,0.000000,5319.409234
22,Sunflower,2,3265.866000,0.000000,35.000074,0.000000,104.999927,0.000000,2001.000000,0.0,1.094313,0.000000,5409.960314
18,Sorghum,2,3356.584500,0.000000,35.000074,0.000000,104.999927,0.000000,2001.000000,0.0,1.509461,0.000000,5501.093962
1,Barley,2,3356.584500,0.000000,35.000074,0.000000,104.999927,0.000000,2001.000000,0.0,2.458498,0.000000,5502.042999
23,Sweet potato,2,4626.643500,0.000000,35.000074,0.000000,104.999927,0.000000,2001.000000,0.0,2.251081,0.000000,6771.894582
9,Groundnut,2,4989.517500,0.000000,35.000074,0.000000,104.999927,0.000000,2001.000000,0.0,2.061786,0.000000,7134.579287


In [37]:
info.sort_values(by=["Water_productivity_avg", "Yield_avg"], ascending= False)

,type,sample_number,Yield_avg,Yield_range,Lat_avg,Lat_range,Long_avg,Long_rang,Altitude_avg,Altitude_range,Water_productivity_avg,Water_productivity_range,sum
20,Sugar beet,2,682475.275500,0.000000,35.000074,0.000000,104.999927,0.000000,2001.000000,0.0,113.398227,0.000000,684731.673728
21,Sugarcane,3,117066.176350,97585.890450,32.084732,8.746025,79.755800,75.732380,1414.666667,1759.0,43.166076,40.599114,218108.817594
15,Potato,45,28734.955090,47944.963500,31.528187,51.023382,53.225060,181.410573,727.715556,1983.1,25.544642,375.415074,80153.881064
26,Tomato,32,42649.034813,117335.307900,2.713628,76.500157,60.742707,141.767741,1104.875000,1721.0,25.222868,37.800000,163186.964813
6,Cucumber,3,57542.744550,7066.971150,40.178600,0.000000,116.444200,0.000000,37.000000,0.0,20.986667,7.700000,64835.025167
4,Cabbage,2,31479.319500,0.000000,35.000074,0.000000,104.999927,0.000000,2001.000000,0.0,18.513996,0.000000,33640.833497
3,Broccoli,17,25765.378235,20868.570000,-23.344853,1.220833,-52.124804,2.971670,536.352941,96.0,13.583452,0.000000,47225.607475
13,Mustard,2,1170.000000,132.000000,23.170152,0.000000,79.932451,0.000000,400.000000,0.0,13.583452,0.000000,1820.686055
10,Haricot bean,12,5950.871903,12347.570496,21.109083,21.787000,34.883935,7.656111,694.416667,1489.0,13.583452,0.000000,20592.878648
8,Green Pea,2,15149.989500,2177.244000,28.613895,0.000000,77.209006,0.000000,214.000000,0.0,12.291726,2.583452,17663.931579


In [43]:
def standardize_plant_type_to_5(plant_type: str):
    if plant_type in ["Sugarcane", "Sugar beet"]:
      return "Sugar"
    if plant_type in ["Barley", "Maize", "Millet", "Rice", "Sorghum", "Wheat"]:
      return "Grains"
    if plant_type in ["Potato", "Sweet potato", "Tomato", "Cucumber", "Cabbage", "Broccoli"]:
      return "Vegetables"
    if plant_type in ["Green Pea", "Haricot bean", "Soybean", "Groundnut"]:
      return "Legumes"
    if plant_type in ["Papaya", "Grapes", "Apple", "Bell pepper"]:
      return "Fruits"
    if plant_type in ["Mustard", "Cotton", "Rapeseed", "Tobacco", "Sunflower", "Tea"]:
      return "Oil / Fiber / Industrial Crops"
    print(plant_type, " unable to be categorized")
    return plant_type

In [44]:
samples["plant_type_standardized_to_5"] = samples["plant_type_standardized"].apply(standardize_plant_type_to_5)

In [46]:
info = []
for name, group in samples.groupby("plant_type_standardized_to_5"):
  info.append({
      'type': name,
      'sample_number': len(group),
      'Yield_avg': group["plant_yield_kg_per_hectare"].sum() / len(group),
      'Yield_range': group["plant_yield_kg_per_hectare"].max() - group["plant_yield_kg_per_hectare"].min(),
      'Lat_avg': group["latitude_decimal_degrees"].sum() / len(group),
      'Lat_range': group["latitude_decimal_degrees"].max() - group["latitude_decimal_degrees"].min(),
      'Long_avg': group["longitude_decimal_degrees"].sum() / len(group),
      'Long_rang': group["longitude_decimal_degrees"].max() - group["longitude_decimal_degrees"].min(),
      'Altitude_avg': group["altitude_numeric"].sum() / len(group),
      'Altitude_range': group["altitude_numeric"].max() - group["altitude_numeric"].min(),
      'Water_productivity_avg': group["water_productivity_kg_per_m^3"].sum() / len(group),
      'Water_productivity_range': group["water_productivity_kg_per_m^3"].max() - group["water_productivity_kg_per_m^3"].min()
  })
info = pd.DataFrame(info)
info["sum"] = info.drop('type', axis= 1).sum(axis = 1)

In [48]:
info.sort_values(by=["sum"])

,type,sample_number,Yield_avg,Yield_range,Lat_avg,Lat_range,Long_avg,Long_rang,Altitude_avg,Altitude_range,Water_productivity_avg,Water_productivity_range,sum
3,Oil / Fiber / Industrial Crops,38,3266.324567,5114.070000,31.762453,26.482153,81.108915,71.480037,923.684211,1885.0,3.880971,13.310452,11455.103760
1,Grains,80,4908.510346,15001.563000,27.811959,36.916700,52.224692,193.819148,898.490000,1998.0,9.684024,14.938267,23221.958136
2,Legumes,37,5306.845698,16222.181996,13.653012,64.831878,-1.652163,190.547698,625.243243,2001.0,10.474046,12.973452,24483.098860
0,Fruits,53,23182.425165,84382.747000,17.678115,63.492775,5.769915,178.705139,864.150943,1729.0,5.794741,11.160000,110493.923794
5,Vegetables,101,33076.175076,119458.120800,13.557209,85.653495,41.803019,247.880259,844.912871,1983.1,22.693559,377.940000,156252.836290
4,Sugar,5,343229.816010,630466.359450,33.250869,8.746025,89.853451,75.732380,1649.200000,1759.0,71.258936,97.298227,977485.515349


In [51]:
columns = ['irrigation_scheduling_method', 'plant_yield',
       'plant_yield_kg_per_hectare', 'plant_yield_notes', 'location',
       'latitude', 'latitude_decimal_degrees', 'latitude_notes', 'lat_filled',
       'longitude', 'longitude_decimal_degrees', 'longitude_notes',
       'long_filled', 'altitude', 'altitude_numeric', 'altitude_notes',
       'altitude_filled', 'plant_type', 'plant_type_standardized', 'plant_type_standardized_to_5',
       'plant_type_notes', 'water_productivity',
       'water_productivity_kg_per_m^3', 'water_productivity_notes',
       'water_productivity_valid', 'water_productivity_filled', 'file_path',
       'doi', 'doi_url', 'processed_at', 'content source', 'relevance_score',]

In [52]:
samples = samples[columns]

In [53]:
samples.to_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files (11 16 25)/Extra analysis/cleaned_samples (9).csv")

# Water Productivity

In [ ]:
samples = pd.read_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (3).csv")
samples.drop("Unnamed: 0", axis=1, inplace=True)
samples

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,longitude,altitude,plant_type,plant_type_standardized,plant_type_notes,water_productivity,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),17.37 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),20.52 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),25.07 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),Wheat,(Triticum aestivum L.),1.42 kg m−3 (70% ETc),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,Broccoli,Brassica oleracea var. italica is the scientif...,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,IW/CPE ratio 0.6,"2595 kg/ha (Wheat), 1104 kg/ha (Mustard)",2595.00000,Yield value for Wheat.,"Jabalpur, Madhya Pradesh, India",NaN,NaN,NaN,Wheat + Mustard,Wheat,Yield value for,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,2025-09-12 23:36:58,pdf,8
278,Non-irrigated,"12,108 kg/ha for corn, 3,569 kg/ha for soybean",12108.00000,Yield value for corn.,Dry corner of each site,NaN,NaN,NaN,"Corn, Soybean",Maize,Yield value for corn.,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9
279,IW/CPE ratio 0.8,"2989 kg/ha (Wheat), 1236 kg/ha (Mustard)",1236.00000,Yield value for Mustard.,"Jabalpur, Madhya Pradesh, India",NaN,NaN,NaN,Wheat + Mustard,Mustard,Yield value for Mustard.,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,2025-09-12 23:36:58,pdf,8
280,IW/CPE ratio 0.6,"2595 kg/ha (Wheat), 1104 kg/ha (Mustard)",1104.00000,Yield value for Mustard.,"Jabalpur, Madhya Pradesh, India",NaN,NaN,NaN,Wheat + Mustard,Mustard,Yield value for Mustard.,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,2025-09-12 23:36:58,pdf,8


In [ ]:
print(len(samples["water_productivity"].unique()))
samples["water_productivity"].unique()

139


array(['17.37 kg m−3', '20.52 kg m−3', '25.07 kg m−3',
       '1.42 kg m−3 (70% ETc)', nan, '508 m3 t-1', '369 m3 t-1',
       '231 m3 t-1', '601 m3 t-1', '719 m3 t-1', '915 m3 t-1',
       '192 m3 t-1', '403 m3 t-1', '2,539 m3 t-1', '16 m3 t-1',
       '8 m3 t-1', '440 m3 t-1', '181 m3 t-1', '829 m3 t-1', '25 m3 t-1',
       '159 m3 t-1', '1,769 m3 t-1', '596 m3 t-1', '49 m3 t-1',
       '135 m3 t-1', '4.32 kg/mm', '6.07 kg/mm', '6.09 kg/mm',
       '5.46 kg/mm', '2.40 kg/ha/mm', '2.31 kg/ha/mm',
       '4.24 kg ha-1mm-1 (2017), 4.04 kg ha-1mm-1 (2018)',
       '5.63 kg ha-1mm-1 (2017), 4.81 kg ha-1mm-1 (2018)', '13.9 kg/m3',
       '380.14 kg/m3', '2.2 kg/m3', '2.39 kg/m3', '16.2 kg/m3',
       '16.1 kg/m3', '0.67 kg/m3', '8.1 kg/m3', '2.09 kg/m3',
       '2.67 kg/m3', '6.81-7.13 kg/m3', '4.94-6.77 kg/m3',
       '4.66-6.05 kg/m3', '4.76-6.17 kg/m3', '4.33-5.61 kg/m3',
       '4.03-5.98 kg/m3', '2.94 kg/m3', '2.76 kg/m3', '2.90 kg/m3',
       '9.19 kg/m3', '7.21 kg/m3', '6.31 kg/m3',

In [ ]:
relative_pattern = r"(increase|mm|c03)"
def is_relative_description(row):
    productivity_value = row["water_productivity"]
    print(productivity_value)
    if type(productivity_value) != str and np.isnan(productivity_value):
      return row
    if bool(re.search(relative_pattern, productivity_value.lower())):
      row["water_productivity"] = np.nan
    return row

# Filter out rows with relative descriptions
samples_clean = samples.apply(is_relative_description, axis= 1).reset_index(drop=True)
samples_clean

Before: 70
17.37 kg m−3
20.52 kg m−3
25.07 kg m−3
1.42 kg m−3 (70% ETc)
nan
nan
nan
nan
1.42 kg m−3 (70% ETc)
508 m3 t-1
369 m3 t-1
231 m3 t-1
601 m3 t-1
719 m3 t-1
369 m3 t-1
915 m3 t-1
192 m3 t-1
403 m3 t-1
2,539 m3 t-1
16 m3 t-1
8 m3 t-1
440 m3 t-1
181 m3 t-1
829 m3 t-1
25 m3 t-1
159 m3 t-1
1,769 m3 t-1
596 m3 t-1
49 m3 t-1
135 m3 t-1
1.42 kg m−3 (70% ETc)
1.42 kg m−3 (70% ETc)
4.32 kg/mm
6.07 kg/mm
6.09 kg/mm
5.46 kg/mm
4.32 kg/mm
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
2.40 kg/ha/mm
2.31 kg/ha/mm
nan
nan
nan
nan
4.24 kg ha-1mm-1 (2017), 4.04 kg ha-1mm-1 (2018)
5.63 kg ha-1mm-1 (2017), 4.81 kg ha-1mm-1 (2018)
13.9 kg/m3
380.14 kg/m3
2.2 kg/m3
2.39 kg/m3
16.2 kg/m3
16.1 kg/m3
0.67 kg/m3
8.1 kg/m3
2.09 kg/m3
nan
2.67 kg/m3
6.81-7.13 kg/m3
4.94-6.77 kg/m3
4.66-6.05 kg/m3
4.76-6.17 kg/m3
4.33-5.61 kg/m3
4.03-5.98 kg/m3
2.94 kg/m3
2.76 kg/m3
2.90 kg/m3
nan
nan
nan
9.19 kg/m3
7.21 kg/m3
6.31 kg/m3
7.16 kg/m3
7.22 kg/m3
6.42 kg/m3
7.10 k

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,longitude,altitude,plant_type,plant_type_standardized,plant_type_notes,water_productivity,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),17.37 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),20.52 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),25.07 kg m−3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),Wheat,(Triticum aestivum L.),1.42 kg m−3 (70% ETc),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,Broccoli,Brassica oleracea var. italica is the scientif...,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,IW/CPE ratio 0.6,"2595 kg/ha (Wheat), 1104 kg/ha (Mustard)",2595.00000,Yield value for Wheat.,"Jabalpur, Madhya Pradesh, India",NaN,NaN,NaN,Wheat + Mustard,Wheat,Yield value for,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,2025-09-12 23:36:58,pdf,8
278,Non-irrigated,"12,108 kg/ha for corn, 3,569 kg/ha for soybean",12108.00000,Yield value for corn.,Dry corner of each site,NaN,NaN,NaN,"Corn, Soybean",Maize,Yield value for corn.,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,2025-09-12 23:44:02,pdf,9
279,IW/CPE ratio 0.8,"2989 kg/ha (Wheat), 1236 kg/ha (Mustard)",1236.00000,Yield value for Mustard.,"Jabalpur, Madhya Pradesh, India",NaN,NaN,NaN,Wheat + Mustard,Mustard,Yield value for Mustard.,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,2025-09-12 23:36:58,pdf,8
280,IW/CPE ratio 0.6,"2595 kg/ha (Wheat), 1104 kg/ha (Mustard)",1104.00000,Yield value for Mustard.,"Jabalpur, Madhya Pradesh, India",NaN,NaN,NaN,Wheat + Mustard,Mustard,Yield value for Mustard.,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,2025-09-12 23:36:58,pdf,8


In [ ]:
print(len(samples_clean["water_productivity"].unique()))
samples_clean["water_productivity"].unique()

108


array(['17.37 kg m−3', '20.52 kg m−3', '25.07 kg m−3',
       '1.42 kg m−3 (70% ETc)', nan, '508 m3 t-1', '369 m3 t-1',
       '231 m3 t-1', '601 m3 t-1', '719 m3 t-1', '915 m3 t-1',
       '192 m3 t-1', '403 m3 t-1', '2,539 m3 t-1', '16 m3 t-1',
       '8 m3 t-1', '440 m3 t-1', '181 m3 t-1', '829 m3 t-1', '25 m3 t-1',
       '159 m3 t-1', '1,769 m3 t-1', '596 m3 t-1', '49 m3 t-1',
       '135 m3 t-1', '13.9 kg/m3', '380.14 kg/m3', '2.2 kg/m3',
       '2.39 kg/m3', '16.2 kg/m3', '16.1 kg/m3', '0.67 kg/m3',
       '8.1 kg/m3', '2.09 kg/m3', '2.67 kg/m3', '6.81-7.13 kg/m3',
       '4.94-6.77 kg/m3', '4.66-6.05 kg/m3', '4.76-6.17 kg/m3',
       '4.33-5.61 kg/m3', '4.03-5.98 kg/m3', '2.94 kg/m3', '2.76 kg/m3',
       '2.90 kg/m3', '9.19 kg/m3', '7.21 kg/m3', '6.31 kg/m3',
       '7.16 kg/m3', '7.22 kg/m3', '6.42 kg/m3', '7.10 kg/m3',
       '4.83 kg/m3', '3.19 kg/m³', '3.07 kg/m³', '2.80 kg/m³',
       '2.75 kg/m³', '2.74 kg/m³', '25.4 kg/m3', '29.2 kg/m3',
       '29.8 kg/m3', '27.4 kg/m3

In [ ]:
class WaterProductivityItem(BaseModel):
  current_unit: str
  reciprocal: bool
  conversion_factor: float
  original_value: float
  cleaning_notes: str

class WaterProductivityLLM(BaseModel):
    water_productivity_list: conlist(WaterProductivityItem, min_items=0) = Field(
        description="List of information for water productivity values"
    )

def standardize_water_productivity(df: pd.DataFrame) -> pd.DataFrame:
    """Convert water_productivity to standardized kg/m^3"""
    print("Standardizing water_productivity...")

    # Get sample values to understand the format
    sample_yields = df['water_productivity'].dropna().head(10).tolist()
    print(f"Sample water productivity values: {sample_yields}")

    prompt = """
You are given water productivity value(s) from agricultural data. Your task is to determine the current unit of these values and how to convert their units to kg/hectare.
If there are multiple comma-separated values, convert and return them all as separate objects.

Analyze the format of each value and provide:
  1. current_unit: The name of the unit this value appears to be in.
  2. reciprocal: true if units are m^3 / t, and false otherwise.
  3. conversion_factor: a float conversion factor from this unit to kg/m^3. If the current unit is kg/m^3, return 1.0.
     Here are some other common conversion factors -
        1 kg = 0.00110231 ton
        1 kg = 1000 g
        1 ha cm = 100 m^3
        1 m3 / t = 0.00110231 m^3 / kg
        1 ton / (ha cm) = 10 kg/m3

  4. original_value: the original float value in the original unit.
  5. cleaning_notes: any extra notes describing value or if there is extra information in parentheses or acronyms
      - If there is a range of values, mention the range and whether this value is a minimum or maximum of that range

Return ONLY as a JSON array of objects.

Water productivity values: {values}
    """

    extra_rows_to_add = []

    # Extract numerical values and convert
    def extract_and_convert_water_productivity(row):
        value = row["water_productivity"]
        if pd.isna(value):
            return row

        # Extract numbers from string
        response = call_gpt_api(llm_type= WaterProductivityLLM, prompt= prompt.format(values=value)).water_productivity_list
        analysis = response[0].dict()
        print("Original value: {}".format(value))
        print(f"First base val {analysis['original_value']}")
        print(analysis)
        if len(response) > 1:
          print("Other values")
          for i in range(1, len(response)):
            val = response[i]
            other_analysis = val.dict()
            print(other_analysis)
            new_row = row.copy()
            if other_analysis["conversion_factor"] is not None:
              # removing rows with unclear units for plant yield
              if analysis["reciprocal"]:
                new_row["water_productivity_kg_per_m^3"] = 1.0 / ( other_analysis["original_value"] * other_analysis["conversion_factor"] )
              else:
                new_row["water_productivity_kg_per_m^3"] = other_analysis["original_value"] * other_analysis["conversion_factor"]
            new_row['water_productivity_notes'] = other_analysis['cleaning_notes']
            extra_rows_to_add.append(new_row)

        if analysis["conversion_factor"] is None:
          row['water_productivity_kg_per_m^3'] = np.nan;
        else:
          if analysis["reciprocal"]:
            row['water_productivity_kg_per_m^3'] = 1.0 / ( analysis["original_value"] * analysis['conversion_factor'] )
          else:
            row['water_productivity_kg_per_m^3'] = analysis["original_value"] * analysis['conversion_factor']
        row['water_productivity_notes'] = analysis['cleaning_notes']

        return row

    df = df.apply(extract_and_convert_water_productivity, axis= 1)

    return df, extra_rows_to_add

In [ ]:
test_samples = samples_clean.loc[samples_clean["water_productivity"].notna()]
test_samples["water_productivity"].unique()

array(['17.37 kg m−3', '20.52 kg m−3', '25.07 kg m−3',
       '1.42 kg m−3 (70% ETc)', '508 m3 t-1', '369 m3 t-1', '231 m3 t-1',
       '601 m3 t-1', '719 m3 t-1', '915 m3 t-1', '192 m3 t-1',
       '403 m3 t-1', '2,539 m3 t-1', '16 m3 t-1', '8 m3 t-1',
       '440 m3 t-1', '181 m3 t-1', '829 m3 t-1', '25 m3 t-1',
       '159 m3 t-1', '1,769 m3 t-1', '596 m3 t-1', '49 m3 t-1',
       '135 m3 t-1', '13.9 kg/m3', '380.14 kg/m3', '2.2 kg/m3',
       '2.39 kg/m3', '16.2 kg/m3', '16.1 kg/m3', '0.67 kg/m3',
       '8.1 kg/m3', '2.09 kg/m3', '2.67 kg/m3', '6.81-7.13 kg/m3',
       '4.94-6.77 kg/m3', '4.66-6.05 kg/m3', '4.76-6.17 kg/m3',
       '4.33-5.61 kg/m3', '4.03-5.98 kg/m3', '2.94 kg/m3', '2.76 kg/m3',
       '2.90 kg/m3', '9.19 kg/m3', '7.21 kg/m3', '6.31 kg/m3',
       '7.16 kg/m3', '7.22 kg/m3', '6.42 kg/m3', '7.10 kg/m3',
       '4.83 kg/m3', '3.19 kg/m³', '3.07 kg/m³', '2.80 kg/m³',
       '2.75 kg/m³', '2.74 kg/m³', '25.4 kg/m3', '29.2 kg/m3',
       '29.8 kg/m3', '27.4 kg/m3', '3

In [ ]:
df, extra_rows_to_add = standardize_water_productivity(samples_clean)
df

Standardizing water_productivity...
Sample water productivity values: ['17.37 kg m−3', '20.52 kg m−3', '25.07 kg m−3', '1.42 kg m−3 (70% ETc)', '1.42 kg m−3 (70% ETc)', '508 m3 t-1', '369 m3 t-1', '231 m3 t-1', '601 m3 t-1', '719 m3 t-1']


/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1900: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


Original value: 17.37 kg m−3
First base val 17.37
{'current_unit': 'kg/m^3', 'reciprocal': False, 'conversion_factor': 1.0, 'original_value': 17.37, 'cleaning_notes': ''}


/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1900: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


Original value: 20.52 kg m−3
First base val 20.52
{'current_unit': 'kg/m^3', 'reciprocal': False, 'conversion_factor': 1.0, 'original_value': 20.52, 'cleaning_notes': ''}


...

/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1900: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


Original value: 0.45-0.71 kg/m3
First base val 0.45
{'current_unit': 'kg/m3', 'reciprocal': False, 'conversion_factor': 1.0, 'original_value': 0.45, 'cleaning_notes': 'Minimum value of the range 0.45-0.71 kg/m3'}
Other values
{'current_unit': 'kg/m3', 'reciprocal': False, 'conversion_factor': 1.0, 'original_value': 0.71, 'cleaning_notes': 'Maximum value of the range 0.45-0.71 kg/m3'}


/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1900: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


Original value: 2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54 kg m−3 (W 1.2) in 2018; 1.58 kg m−3 (W 1.2) in 2019
First base val 2.53
{'current_unit': 'kg/m^3', 'reciprocal': False, 'conversion_factor': 1.0, 'original_value': 2.53, 'cleaning_notes': '(W 0.6)'}
Other values
{'current_unit': 'kg/m^3', 'reciprocal': False, 'conversion_factor': 1.0, 'original_value': 2.6, 'cleaning_notes': '(W 0.8)'}
{'current_unit': 'kg/m^3', 'reciprocal': False, 'conversion_factor': 1.0, 'original_value': 1.54, 'cleaning_notes': '(W 1.2)'}
{'current_unit': 'kg/m^3', 'reciprocal': False, 'conversion_factor': 1.0, 'original_value': 1.58, 'cleaning_notes': '(W 1.2)'}


,altitude,content source,doi,doi_url,file_path,irrigation_scheduling_method,latitude,location,longitude,plant_type,plant_type_notes,plant_type_standardized,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,processed_at,relevance_score,water_productivity,water_productivity_kg_per_m^3,water_productivity_notes
0,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Soil moisture sensor-based method (T-VWC),40.1786° N,Xiaotangshan National Precision Agriculture Ex...,116.4442° E,Cucumber (Zhongnong 26),(Zhongnong 26),Cucumber,66.77 t ha−1,60572.74245,This value is given in tons per hectare.,2025-09-12 01:42:16,9,17.37 kg m−3,17.37,
1,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Accumulated radiation combined with soil moist...,40.1786° N,Xiaotangshan National Precision Agriculture Ex...,116.4442° E,Cucumber (Zhongnong 26),(Zhongnong 26),Cucumber,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,2025-09-12 01:42:16,9,20.52 kg m−3,20.52,
2,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Crop evapotranspiration estimated method using...,40.1786° N,Xiaotangshan National Precision Agriculture Ex...,116.4442° E,Cucumber (Zhongnong 26),(Zhongnong 26),Cucumber,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,2025-09-12 01:42:16,9,25.07 kg m−3,25.07,
3,1550 m,pdf,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Soil water balance method,8°24′N,"Melkassa Agricultural Research Center, Awash B...",39°21′E,Wheat (Triticum aestivum L.),(Triticum aestivum L.),Wheat,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,2025-09-12 04:01:02,9,1.42 kg m−3 (70% ETc),1.42,(70% ETc)
4,542 m,pdf,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Regulated Deficit Irrigation,23º25′S,"Maringá, Paraná, Brazil",51º57′W,Brassica oleracea var. italica,Brassica oleracea var. italica is the scientif...,Broccoli,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...",2025-09-12 04:06:41,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,NaN,pdf,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,IW/CPE ratio 0.6,NaN,"Jabalpur, Madhya Pradesh, India",NaN,Wheat + Mustard,Yield value for,Wheat,"2595 kg/ha (Wheat), 1104 kg/ha (Mustard)",2595.00000,Yield value for Wheat.,2025-09-12 23:36:58,8,NaN,NaN,NaN
278,NaN,pdf,https://doi.org/10.3389/fagro.2025.1496198,https://doi.org/10.3389/fagro.2025.1496198,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Non-irrigated,NaN,Dry corner of each site,NaN,"Corn, Soybean",Yield value for corn.,Maize,"12,108 kg/ha for corn, 3,569 kg/ha for soybean",12108.00000,Yield value for corn.,2025-09-12 23:44:02,9,NaN,NaN,NaN
279,NaN,pdf,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,IW/CPE ratio 0.8,NaN,"Jabalpur, Madhya Pradesh, India",NaN,Wheat + Mustard,Yield value for Mustard.,Mustard,"2989 kg/ha (Wheat), 1236 kg/ha (Mustard)",1236.00000,Yield value for Mustard.,2025-09-12 23:36:58,8,NaN,NaN,NaN
280,NaN,pdf,http://dx.doi.org/10.5281/zenodo.7790864,http://dx.doi.org/10.5281/zenodo.7790864,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,IW/CPE ratio 0.6,NaN,"Jabalpur, Madhya Pradesh, India",NaN,Wheat + Mustard,Yield value for Mustard.,Mustard,"2595 kg/ha (Wheat), 1104 kg/ha (Mustard)",1104.00000,Yield value for Mustard.,2025-09-12 23:36:58,8,NaN,NaN,NaN


In [ ]:
pd.DataFrame(extra_rows_to_add)

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,longitude,altitude,plant_type,plant_type_standardized,plant_type_notes,water_productivity,file_path,doi,doi_url,processed_at,content source,relevance_score,water_productivity_kg_per_m^3,water_productivity_notes
79,Rainfed,5.3-14.1 t/ha,4808.08050,NaN,"Fuente-Álamo, Albacete, Spain",38°43′ N,1°28′ W,820 m,Vitis vinifera L. cv. Monastrell,Grapes,Vitis vinifera L. cv. Monastrell,6.81-7.13 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1007/s00271-022-00795-x,https://doi.org/10.1007/s00271-022-00795-x,2025-09-12 05:15:05,pdf,9,7.130,Maximum value of the range 6.81-7.13 kg/m3
80,Control (standard water),8.8-21.3 t/ha,7983.22800,NaN,"Fuente-Álamo, Albacete, Spain",38°43′ N,1°28′ W,820 m,Vitis vinifera L. cv. Monastrell,Grapes,Vitis vinifera L. cv. Monastrell,4.94-6.77 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1007/s00271-022-00795-x,https://doi.org/10.1007/s00271-022-00795-x,2025-09-12 05:15:05,pdf,9,6.770,Maximum value of the range 4.94-6.77 kg/m3
81,Sulphate treatment (Control + Sulphates),8.4-18.9 t/ha,7620.35400,NaN,"Fuente-Álamo, Albacete, Spain",38°43′ N,1°28′ W,820 m,Vitis vinifera L. cv. Monastrell,Grapes,Vitis vinifera L. cv. Monastrell,4.66-6.05 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1007/s00271-022-00795-x,https://doi.org/10.1007/s00271-022-00795-x,2025-09-12 05:15:05,pdf,9,6.050,Maximum value of the range 4.66-6.05 kg/m3
82,Chloride treatment (Control + Chloride),8.4-19.3 t/ha,7620.35400,NaN,"Fuente-Álamo, Albacete, Spain",38°43′ N,1°28′ W,820 m,Vitis vinifera L. cv. Monastrell,Grapes,Vitis vinifera L. cv. Monastrell,4.76-6.17 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1007/s00271-022-00795-x,https://doi.org/10.1007/s00271-022-00795-x,2025-09-12 05:15:05,pdf,9,6.170,Maximum value of the range 4.76-6.17 kg/m3
83,Sulphate after veraison treatment (Control + S...,7.9-17.6 t/ha,7166.76150,Lower end of the yield range.,"Fuente-Álamo, Albacete, Spain",38°43′ N,1°28′ W,820 m,Vitis vinifera L. cv. Monastrell,Grapes,Vitis vinifera L. cv. Monastrell,4.33-5.61 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1007/s00271-022-00795-x,https://doi.org/10.1007/s00271-022-00795-x,2025-09-12 05:15:05,pdf,9,5.610,Maximum value of the range 4.33-5.61 kg/m3
84,Chloride after veraison treatment (Control + C...,7.1-18.2 t/ha,6441.01350,NaN,"Fuente-Álamo, Albacete, Spain",38°43′ N,1°28′ W,820 m,Vitis vinifera L. cv. Monastrell,Grapes,Vitis vinifera L. cv. Monastrell,4.03-5.98 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1007/s00271-022-00795-x,https://doi.org/10.1007/s00271-022-00795-x,2025-09-12 05:15:05,pdf,9,5.980,Maximum value of the range 4.03-5.98 kg/m3
159,Pre-sowing irrigation (PSI) 120 mm,"2549 kg ha-1 (2016), 2040 kg ha-1 (2017)",2549.00000,Yield value for the year 2016.,"Punjab Agricultural University, Bathinda, Punj...",30.09°N,74.55°E,211 m,Cotton (Gossypium hirsutum L.),Cotton,(Gossypium hirsutum L.),"RCWP: 458 g m-3 (2016), 376 g m-3 (2017); TCWP...",/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1016/j.agwat.2022.107982,https://doi.org/10.1016/j.agwat.2022.107982,2025-09-12 14:23:48,pdf,9,0.376,2017 value for RCWP
159,Pre-sowing irrigation (PSI) 120 mm,"2549 kg ha-1 (2016), 2040 kg ha-1 (2017)",2549.00000,Yield value for the year 2016.,"Punjab Agricultural University, Bathinda, Punj...",30.09°N,74.55°E,211 m,Cotton (Gossypium hirsutum L.),Cotton,(Gossypium hirsutum L.),"RCWP: 458 g m-3 (2016), 376 g m-3 (2017); TCWP...",/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1016/j.agwat.2022.107982,https://doi.org/10.1016/j.agwat.2022.107982,2025-09-12 14:23:48,pdf,9,0.283,2016 value for TCWP
159,Pre-sowing irrigation (PSI) 120 mm,"2549 kg ha-1 (2016), 2040 kg ha-1 (2017)",2549.00000,Yield value for the year 

In [ ]:
combined_samples = pd.concat([df, pd.DataFrame(extra_rows_to_add)], ignore_index=True)
combined_samples

,altitude,content source,doi,doi_url,file_path,irrigation_scheduling_method,latitude,location,longitude,plant_type,plant_type_notes,plant_type_standardized,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,processed_at,relevance_score,water_productivity,water_productivity_kg_per_m^3,water_productivity_notes
0,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Soil moisture sensor-based method (T-VWC),40.1786° N,Xiaotangshan National Precision Agriculture Ex...,116.4442° E,Cucumber (Zhongnong 26),(Zhongnong 26),Cucumber,66.77 t ha−1,60572.74245,This value is given in tons per hectare.,2025-09-12 01:42:16,9,17.37 kg m−3,17.37,
1,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Accumulated radiation combined with soil moist...,40.1786° N,Xiaotangshan National Precision Agriculture Ex...,116.4442° E,Cucumber (Zhongnong 26),(Zhongnong 26),Cucumber,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,2025-09-12 01:42:16,9,20.52 kg m−3,20.52,
2,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Crop evapotranspiration estimated method using...,40.1786° N,Xiaotangshan National Precision Agriculture Ex...,116.4442° E,Cucumber (Zhongnong 26),(Zhongnong 26),Cucumber,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,2025-09-12 01:42:16,9,25.07 kg m−3,25.07,
3,1550 m,pdf,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Soil water balance method,8°24′N,"Melkassa Agricultural Research Center, Awash B...",39°21′E,Wheat (Triticum aestivum L.),(Triticum aestivum L.),Wheat,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,2025-09-12 04:01:02,9,1.42 kg m−3 (70% ETc),1.42,(70% ETc)
4,542 m,pdf,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Regulated Deficit Irrigation,23º25′S,"Maringá, Paraná, Brazil",51º57′W,Brassica oleracea var. italica,Brassica oleracea var. italica is the scientif...,Broccoli,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...",2025-09-12 04:06:41,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,521 m,pdf,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"100% ETc, 4-day interval",34°20′N,"Yangling, Shaanxi, China",108°04′E,Solanum lycopersicum L.,Solanum lycopersicum L.,Tomato,"136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,2025-09-12 17:23:02,10,"32.7 kg/m3 (Spring 2023), 19.3 kg/m3 (Autumn 2...",19.30,Autumn 2023
312,NaN,pdf,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Timing of last irrigation,NaN,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,Cotton (Gossypium hirsutum L.),(Gossypium hirsutum L.),Cotton,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,2025-09-12 23:58:14,8,0.45-0.71 kg/m3,0.71,Maximum value of the range 0.45-0.71 kg/m3
313,900 m,pdf,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Film-Mulched Drip Irrigation,40°53′N,"Korla, Xinjiang, China",86°56′E,Gossypium hirstum L.,Gossypium hirstum L. is a scientific name for ...,Cotton,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,2025-09-13 00:18:02,9,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",2.60,(W 0.8)
314,900 m,pdf,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,/content/drive/MyDrive/1:1_Sruthi_Manoj/

In [ ]:
combined_samples = combined_samples[["irrigation_scheduling_method", "plant_yield", "plant_yield_kg_per_hectare", "plant_yield_notes",
                  "location", "latitude", "longitude", "altitude", "plant_type", "plant_type_standardized", "plant_type_notes",
                  "water_productivity", "water_productivity_kg_per_m^3", "water_productivity_notes", "file_path", "doi", "doi_url", "processed_at", "content source", "relevance_score"]]
combined_samples

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,longitude,altitude,plant_type,plant_type_standardized,plant_type_notes,water_productivity,water_productivity_kg_per_m^3,water_productivity_notes,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),17.37 kg m−3,17.37,,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),20.52 kg m−3,20.52,,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),25.07 kg m−3,25.07,,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),Wheat,(Triticum aestivum L.),1.42 kg m−3 (70% ETc),1.42,(70% ETc),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,Broccoli,Brassica oleracea var. italica is the scientif...,NaN,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,"100% ETc, 4-day interval","136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,108°04′E,521 m,Solanum lycopersicum L.,Tomato,Solanum lycopersicum L.,"32.7 kg/m3 (Spring 2023), 19.3 kg/m3 (Autumn 2...",19.30,Autumn 2023,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10
312,Timing of last irrigation,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,NaN,NaN,Cotton (Gossypium hirsutum L.),Cotton,(Gossypium hirsutum L.),0.45-0.71 kg/m3,0.71,Maximum value of the range 0.45-0.71 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8
313,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,86°56′E,900 m,Gossypium hirstum L.,Cotton,Gossypium hirstum L. is a scientific name for ...,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",2.60,(W 0.8),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9
314,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinj

In [ ]:
combined_samples.to_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (4).csv")

(10/3/25) Filering using notes prompt

In [ ]:
samples = pd.read_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (4).csv")
samples.drop("Unnamed: 0", axis= 1, inplace= True)
samples

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,longitude,altitude,plant_type,plant_type_standardized,plant_type_notes,water_productivity,water_productivity_kg_per_m^3,water_productivity_notes,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),17.37 kg m−3,17.37,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),20.52 kg m−3,20.52,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,(Zhongnong 26),25.07 kg m−3,25.07,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),Wheat,(Triticum aestivum L.),1.42 kg m−3 (70% ETc),1.42,(70% ETc),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,Broccoli,Brassica oleracea var. italica is the scientif...,NaN,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,"100% ETc, 4-day interval","136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,108°04′E,521 m,Solanum lycopersicum L.,Tomato,Solanum lycopersicum L.,"32.7 kg/m3 (Spring 2023), 19.3 kg/m3 (Autumn 2...",19.30,Autumn 2023,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10
312,Timing of last irrigation,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,NaN,NaN,Cotton (Gossypium hirsutum L.),Cotton,(Gossypium hirsutum L.),0.45-0.71 kg/m3,0.71,Maximum value of the range 0.45-0.71 kg/m3,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8
313,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,86°56′E,900 m,Gossypium hirstum L.,Cotton,Gossypium hirstum L. is a scientific name for ...,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",2.60,(W 0.8),/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9
314,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Ko

In [ ]:
class WaterProductivityItem(BaseModel):
  valid: bool

class WaterProductivityLLM(BaseModel):
    water_productivity_list: conlist(WaterProductivityItem, min_items=0) = Field(
        description="list of whether a water productivity value is valid or not"
    )

def standardize_water_productivity_notes(df: pd.DataFrame) -> pd.DataFrame:
    """Convert water_productivity to standardized kg/m^3"""
    print("Standardizing water_productivity...")

    # Get sample values to understand the format
    sample_values = df['water_productivity'].dropna().head(10).tolist()
    print(f"Sample water productivity values: {sample_values}")

    notes_prompt = """
You are given water productivity cleaning notes and other general requirements notes after processing agricultural data.

Task: Analyze the general notes below and compare to water productivity cleaning notes
  1. Check if the general notes contains information of a type similar to anything in the water productivity cleaning notes (E.g. year, scheduling method, plant type, evapotranspiration level, W, etc.)
      - Units are NOT valid information types. Do not compare them.
      - Choose a specific information type, rather than some letters, with W as an exception. W measures soil moisture content and field capacity.
  2. If there are similar, valid information types, check if the information content matches. (E.g. if both notes have same years, type, evapotranspiration level, etc.)

If the information matches or if there are no matching information types, return the following:
  valid: True

Only if there is information of the same category and they do not match, return the following:
  valid: False

Return ONLY as a JSON array of objects.

Water productivity cleaning notes: {cleaning_notes}
General notes: {general_notes}
    """

    extra_rows_to_add = []

    # Extract numerical values and convert
    def extract_and_convert_water_productivity_notes(row):
        notes = row["water_productivity_notes"]
        general_notes = str(row["plant_yield_notes"] or "") + " " + str(row["plant_type_notes"] or "")
        if pd.isna(notes) or len(general_notes) < 2:
            return row

        # Extract numbers from string
        response = call_gpt_api(llm_type= WaterProductivityLLM, prompt= notes_prompt.format(cleaning_notes=notes, general_notes=general_notes)).water_productivity_list
        analysis = response[0].dict()
        print("Cleaning notes: {}".format(notes))
        print("General notes: {}".format(general_notes))
        print(analysis)
        if len(response) > 1:
          print("Other values")
          for i in range(1, len(response)):
            val = response[i]
            other_analysis = val.dict()
            print(other_analysis)
            new_row = row.copy()
            new_row['valid'] = other_analysis['valid']
            extra_rows_to_add.append(new_row)

        row['valid'] = analysis['valid']

        return row

    df = df.apply(extract_and_convert_water_productivity_notes, axis= 1)

    return df, extra_rows_to_add

In [ ]:
df, extra_rows_to_add = standardize_water_productivity_notes(samples)
df

Standardizing water_productivity...
Sample water productivity values: ['17.37 kg m−3', '20.52 kg m−3', '25.07 kg m−3', '1.42 kg m−3 (70% ETc)', '1.42 kg m−3 (70% ETc)', '508 m3 t-1', '369 m3 t-1', '231 m3 t-1', '601 m3 t-1', '719 m3 t-1']
Cleaning notes: (70% ETc)
General notes: 100% ETc (Triticum aestivum L.)
{'valid': False}
Cleaning notes: Includes 70% ETc
General notes: 100% ETc (Triticum aestivum L.)
{'valid': False}
Cleaning notes: (70% ETc)
General notes: 100% ETc (Triticum aestivum L.)
{'valid': False}
Cleaning notes: (70% ETc)
General notes: 100% ETc (Triticum aestivum L.)
{'valid': False}
Cleaning notes: Minimum value of the range 6.81-7.13 kg/m3
General notes: nan Vitis vinifera L. cv. Monastrell
{'valid': True}
Cleaning notes: Minimum value of the range 4.94-6.77 kg/m3
General notes: nan Vitis vinifera L. cv. Monastrell
{'valid': True}
Cleaning notes: Minimum value of the range 4.66-6.05 kg/m3
General notes: nan Vitis vinifera L. cv. Monastrell
{'valid': True}
Cleaning note

,altitude,content source,doi,doi_url,file_path,irrigation_scheduling_method,latitude,location,longitude,plant_type,...,plant_type_standardized,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,processed_at,relevance_score,valid,water_productivity,water_productivity_kg_per_m^3,water_productivity_notes
0,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Soil moisture sensor-based method (T-VWC),40.1786° N,Xiaotangshan National Precision Agriculture Ex...,116.4442° E,Cucumber (Zhongnong 26),...,Cucumber,66.77 t ha−1,60572.74245,This value is given in tons per hectare.,2025-09-12 01:42:16,9,NaN,17.37 kg m−3,17.37,NaN
1,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Accumulated radiation combined with soil moist...,40.1786° N,Xiaotangshan National Precision Agriculture Ex...,116.4442° E,Cucumber (Zhongnong 26),...,Cucumber,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,2025-09-12 01:42:16,9,NaN,20.52 kg m−3,20.52,NaN
2,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Crop evapotranspiration estimated method using...,40.1786° N,Xiaotangshan National Precision Agriculture Ex...,116.4442° E,Cucumber (Zhongnong 26),...,Cucumber,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,2025-09-12 01:42:16,9,NaN,25.07 kg m−3,25.07,NaN
3,1550 m,pdf,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Soil water balance method,8°24′N,"Melkassa Agricultural Research Center, Awash B...",39°21′E,Wheat (Triticum aestivum L.),...,Wheat,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,2025-09-12 04:01:02,9,False,1.42 kg m−3 (70% ETc),1.42,(70% ETc)
4,542 m,pdf,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Regulated Deficit Irrigation,23º25′S,"Maringá, Paraná, Brazil",51º57′W,Brassica oleracea var. italica,...,Broccoli,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...",2025-09-12 04:06:41,9,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,521 m,pdf,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"100% ETc, 4-day interval",34°20′N,"Yangling, Shaanxi, China",108°04′E,Solanum lycopersicum L.,...,Tomato,"136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,2025-09-12 17:23:02,10,True,"32.7 kg/m3 (Spring 2023), 19.3 kg/m3 (Autumn 2...",19.30,Autumn 2023
312,NaN,pdf,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Timing of last irrigation,NaN,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,Cotton (Gossypium hirsutum L.),...,Cotton,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,2025-09-12 23:58:14,8,True,0.45-0.71 kg/m3,0.71,Maximum value of the range 0.45-0.71 kg/m3
313,900 m,pdf,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Film-Mulched Drip Irrigation,40°53′N,"Korla, Xinjiang, China",86°56′E,Gossypium hirstum L.,...,Cotton,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,2025-09-13 00:18:02,9,True,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",2.60,(W 0.8)
314,900 m,pdf,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Film-Mulched Drip Irrigation,40°53′N,"Korla, Xinjiang, China",86°56′E,Gossypium hirstum L.,...,Cotton,"6218.07 kg ha−1 (2018), 5660

In [ ]:
len(df.loc[df["valid"] == False, "water_productivity_kg_per_m^3"])

19

In [ ]:
df

,altitude,content source,doi,doi_url,file_path,irrigation_scheduling_method,latitude,location,longitude,plant_type,...,plant_type_standardized,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,processed_at,relevance_score,valid,water_productivity,water_productivity_kg_per_m^3,water_productivity_notes
0,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Soil moisture sensor-based method (T-VWC),40.1786° N,Xiaotangshan National Precision Agriculture Ex...,116.4442° E,Cucumber (Zhongnong 26),...,Cucumber,66.77 t ha−1,60572.74245,This value is given in tons per hectare.,2025-09-12 01:42:16,9,NaN,17.37 kg m−3,17.37,NaN
1,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Accumulated radiation combined with soil moist...,40.1786° N,Xiaotangshan National Precision Agriculture Ex...,116.4442° E,Cucumber (Zhongnong 26),...,Cucumber,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,2025-09-12 01:42:16,9,NaN,20.52 kg m−3,20.52,NaN
2,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Crop evapotranspiration estimated method using...,40.1786° N,Xiaotangshan National Precision Agriculture Ex...,116.4442° E,Cucumber (Zhongnong 26),...,Cucumber,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,2025-09-12 01:42:16,9,NaN,25.07 kg m−3,25.07,NaN
3,1550 m,pdf,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Soil water balance method,8°24′N,"Melkassa Agricultural Research Center, Awash B...",39°21′E,Wheat (Triticum aestivum L.),...,Wheat,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,2025-09-12 04:01:02,9,False,1.42 kg m−3 (70% ETc),NaN,(70% ETc)
4,542 m,pdf,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Regulated Deficit Irrigation,23º25′S,"Maringá, Paraná, Brazil",51º57′W,Brassica oleracea var. italica,...,Broccoli,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...",2025-09-12 04:06:41,9,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,521 m,pdf,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"100% ETc, 4-day interval",34°20′N,"Yangling, Shaanxi, China",108°04′E,Solanum lycopersicum L.,...,Tomato,"136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,2025-09-12 17:23:02,10,True,"32.7 kg/m3 (Spring 2023), 19.3 kg/m3 (Autumn 2...",19.30,Autumn 2023
312,NaN,pdf,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Timing of last irrigation,NaN,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,Cotton (Gossypium hirsutum L.),...,Cotton,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,2025-09-12 23:58:14,8,True,0.45-0.71 kg/m3,0.71,Maximum value of the range 0.45-0.71 kg/m3
313,900 m,pdf,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Film-Mulched Drip Irrigation,40°53′N,"Korla, Xinjiang, China",86°56′E,Gossypium hirstum L.,...,Cotton,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,2025-09-13 00:18:02,9,True,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",2.60,(W 0.8)
314,900 m,pdf,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Film-Mulched Drip Irrigation,40°53′N,"Korla, Xinjiang, China",86°56′E,Gossypium hirstum L.,...,Cotton,"6218.07 kg ha−1 (2018), 5660.

In [ ]:
df.rename({'valid' : 'water_productivity_valid'}, axis= 1, inplace= True)
combined_samples = df[["irrigation_scheduling_method", "plant_yield", "plant_yield_kg_per_hectare", "plant_yield_notes",
                  "location", "latitude", "longitude", "altitude", "plant_type", "plant_type_standardized", "plant_type_notes",
                  "water_productivity", "water_productivity_kg_per_m^3", "water_productivity_notes", "water_productivity_valid", "file_path", "doi", "doi_url", "processed_at", "content source", "relevance_score"]]
combined_samples

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,longitude,altitude,plant_type,plant_type_standardized,...,water_productivity,water_productivity_kg_per_m^3,water_productivity_notes,water_productivity_valid,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,...,17.37 kg m−3,17.37,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,...,20.52 kg m−3,20.52,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,...,25.07 kg m−3,25.07,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),Wheat,...,1.42 kg m−3 (70% ETc),NaN,(70% ETc),False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,Broccoli,...,NaN,NaN,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,"100% ETc, 4-day interval","136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,108°04′E,521 m,Solanum lycopersicum L.,Tomato,...,"32.7 kg/m3 (Spring 2023), 19.3 kg/m3 (Autumn 2...",19.30,Autumn 2023,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10
312,Timing of last irrigation,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,NaN,NaN,Cotton (Gossypium hirsutum L.),Cotton,...,0.45-0.71 kg/m3,0.71,Maximum value of the range 0.45-0.71 kg/m3,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8
313,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,86°56′E,900 m,Gossypium hirstum L.,Cotton,...,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",2.60,(W 0.8),True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9
314,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,86°56′E,900 m,Gossypium hirstum L.,Cotton,...,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",1.54

In [ ]:
combined_samples.to_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (5).csv")

# Latitude

In [ ]:
samples = pd.read_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (5).csv")
samples.drop("Unnamed: 0", axis= True, inplace=True)
samples

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,longitude,altitude,plant_type,plant_type_standardized,...,water_productivity,water_productivity_kg_per_m^3,water_productivity_notes,water_productivity_valid,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,...,17.37 kg m−3,17.37,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,...,20.52 kg m−3,20.52,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,116.4442° E,NaN,Cucumber (Zhongnong 26),Cucumber,...,25.07 kg m−3,25.07,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,39°21′E,1550 m,Wheat (Triticum aestivum L.),Wheat,...,1.42 kg m−3 (70% ETc),NaN,(70% ETc),False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,51º57′W,542 m,Brassica oleracea var. italica,Broccoli,...,NaN,NaN,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,"100% ETc, 4-day interval","136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,108°04′E,521 m,Solanum lycopersicum L.,Tomato,...,"32.7 kg/m3 (Spring 2023), 19.3 kg/m3 (Autumn 2...",19.30,Autumn 2023,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10
312,Timing of last irrigation,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,NaN,NaN,Cotton (Gossypium hirsutum L.),Cotton,...,0.45-0.71 kg/m3,0.71,Maximum value of the range 0.45-0.71 kg/m3,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8
313,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,86°56′E,900 m,Gossypium hirstum L.,Cotton,...,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",2.60,(W 0.8),True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9
314,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,86°56′E,900 m,Gossypium hirstum L.,Cotton,...,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",1.54

In [ ]:
class LatitudeItem(BaseModel):
  latitude: float
  cleaning_notes: str

def standardize_latitude(df: pd.DataFrame) -> pd.DataFrame:
    """Convert latitude to decimal degrees"""
    print("Standardizing latitude...")

    # Get sample values to understand the format
    sample_values = df['latitude'].dropna().head(10).tolist()
    print(f"Sample latitude values: {sample_values}")

    prompt = """
You are given latitude values.

Task: Convert the given latitude values to decimal degrees, and return:
  1. latitude: the decimal degrees value
  2. cleaning_notes: any cleaning notes or extra information in original latitude value

Return ONLY as a JSON array of objects.

Latitude value: {value}
    """

    # Extract numerical values and convert
    def extract_and_convert_latitude(row):
        value = row["latitude"]
        if pd.isna(value):
            return row

        # Extract numbers from string
        response = call_gpt_api(llm_type= LatitudeItem, prompt= prompt.format(value=value))
        analysis = response.dict()
        print("Original value: {}".format(value))
        print(analysis)

        row['latitude_decimal_degrees'] = analysis['latitude']
        row['latitude_notes'] = analysis['cleaning_notes']

        return row

    df = df.apply(extract_and_convert_latitude, axis= 1)

    return df

In [ ]:
df = standardize_latitude(samples)
df

Standardizing latitude...
Sample latitude values: ['40.1786° N', '40.1786° N', '40.1786° N', '8°24′N', '23º25′S', '23º25′S', '23º25′S', '23º25′S', '8°24′N', '8°24′N']
Original value: 40.1786° N
{'latitude': 40.1786, 'cleaning_notes': 'Original value was in degrees with a directional indicator (N)'}
Original value: 40.1786° N
{'latitude': 40.1786, 'cleaning_notes': 'Original value was in degrees with a directional indicator (N)'}
Original value: 40.1786° N
{'latitude': 40.1786, 'cleaning_notes': 'Original value was in degrees with a directional indicator (N)'}
Original value: 8°24′N
{'latitude': 8.4, 'cleaning_notes': 'Converted from degrees and minutes format (8°24′N) to decimal degrees.'}
Original value: 23º25′S
{'latitude': -23.416666666666668, 'cleaning_notes': 'Converted from degrees and minutes format (23º25′S) to decimal degrees.'}
Original value: 23º25′S
{'latitude': -23.416666666666668, 'cleaning_notes': 'Converted from degrees and minutes (23º25′S) to decimal degrees.'}
Origin

,altitude,content source,doi,doi_url,file_path,irrigation_scheduling_method,latitude,latitude_decimal_degrees,latitude_notes,location,...,plant_type_standardized,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,processed_at,relevance_score,water_productivity,water_productivity_kg_per_m^3,water_productivity_notes,water_productivity_valid
0,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Soil moisture sensor-based method (T-VWC),40.1786° N,40.178600,Original value was in degrees with a direction...,Xiaotangshan National Precision Agriculture Ex...,...,Cucumber,66.77 t ha−1,60572.74245,This value is given in tons per hectare.,2025-09-12 01:42:16,9,17.37 kg m−3,17.37,NaN,NaN
1,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Accumulated radiation combined with soil moist...,40.1786° N,40.178600,Original value was in degrees with a direction...,Xiaotangshan National Precision Agriculture Ex...,...,Cucumber,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,2025-09-12 01:42:16,9,20.52 kg m−3,20.52,NaN,NaN
2,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Crop evapotranspiration estimated method using...,40.1786° N,40.178600,Original value was in degrees with a direction...,Xiaotangshan National Precision Agriculture Ex...,...,Cucumber,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,2025-09-12 01:42:16,9,25.07 kg m−3,25.07,NaN,NaN
3,1550 m,pdf,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Soil water balance method,8°24′N,8.400000,Converted from degrees and minutes format (8°2...,"Melkassa Agricultural Research Center, Awash B...",...,Wheat,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,2025-09-12 04:01:02,9,1.42 kg m−3 (70% ETc),NaN,(70% ETc),False
4,542 m,pdf,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Regulated Deficit Irrigation,23º25′S,-23.416667,Converted from degrees and minutes format (23º...,"Maringá, Paraná, Brazil",...,Broccoli,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...",2025-09-12 04:06:41,9,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,521 m,pdf,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"100% ETc, 4-day interval",34°20′N,34.333333,Original value was in degrees and minutes form...,"Yangling, Shaanxi, China",...,Tomato,"136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,2025-09-12 17:23:02,10,"32.7 kg/m3 (Spring 2023), 19.3 kg/m3 (Autumn 2...",19.30,Autumn 2023,True
312,NaN,pdf,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Timing of last irrigation,NaN,NaN,NaN,"Gezira Research Station Farm, Wad Medani, Sudan",...,Cotton,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,2025-09-12 23:58:14,8,0.45-0.71 kg/m3,0.71,Maximum value of the range 0.45-0.71 kg/m3,True
313,900 m,pdf,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Film-Mulched Drip Irrigation,40°53′N,40.883333,Original value was in degrees and minutes form...,"Korla, Xinjiang, China",...,Cotton,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,2025-09-13 00:18:02,9,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",2.60,(W 0.8),True
314,900 m,pdf,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy150716

# Longitude

In [ ]:
class LongitudeItem(BaseModel):
  longitude: float
  cleaning_notes: str

def standardize_longitude(df: pd.DataFrame) -> pd.DataFrame:
    """Convert longitude to decimal degrees"""
    print("Standardizing longitude...")

    # Get sample values to understand the format
    sample_values = df['longitude'].dropna().head(10).tolist()
    print(f"Sample longitude values: {sample_values}")

    prompt = """
You are given longitude values.

Task: Convert the given longitude values to decimal degrees, and return:
  1. longitude: the decimal degrees value
  2. cleaning_notes: any cleaning notes or extra information in original longitude value

Return ONLY as a JSON array of objects.

Longitude value: {value}
    """

    # Extract numerical values and convert
    def extract_and_convert_longitude(row):
        value = row["longitude"]
        if pd.isna(value):
            return row

        # Extract numbers from string
        response = call_gpt_api(llm_type= LongitudeItem, prompt= prompt.format(value=value))
        analysis = response.dict()
        print("Original value: {}".format(value))
        print(analysis)

        row['longitude_decimal_degrees'] = analysis['longitude']
        row['longitude_notes'] = analysis['cleaning_notes']

        return row

    df = df.apply(extract_and_convert_longitude, axis= 1)

    return df

In [ ]:
next_df = standardize_longitude(df)
next_df

Standardizing longitude...
Sample longitude values: ['116.4442° E', '116.4442° E', '116.4442° E', '39°21′E', '51º57′W', '51º57′W', '51º57′W', '51º57′W', '39°21′E', '39°21′E']
Original value: 116.4442° E
{'longitude': 116.4442, 'cleaning_notes': 'Original value in degrees with direction E'}
Original value: 116.4442° E
{'longitude': 116.4442, 'cleaning_notes': ''}
Original value: 116.4442° E
{'longitude': 116.4442, 'cleaning_notes': ''}
Original value: 39°21′E
{'longitude': 39.35, 'cleaning_notes': 'Converted from degrees and minutes to decimal degrees.'}
Original value: 51º57′W
{'longitude': -51.95, 'cleaning_notes': 'Converted from degrees and minutes to decimal degrees.'}
Original value: 51º57′W
{'longitude': -51.95, 'cleaning_notes': 'Converted from degrees, minutes to decimal degrees.'}
Original value: 51º57′W
{'longitude': -51.95, 'cleaning_notes': 'Converted from degrees, minutes format (51º57′W) to decimal degrees.'}
Original value: 51º57′W
{'longitude': -51.95, 'cleaning_notes':

,altitude,content source,doi,doi_url,file_path,irrigation_scheduling_method,latitude,latitude_decimal_degrees,latitude_notes,location,...,plant_type_standardized,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,processed_at,relevance_score,water_productivity,water_productivity_kg_per_m^3,water_productivity_notes,water_productivity_valid
0,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Soil moisture sensor-based method (T-VWC),40.1786° N,40.178600,Original value was in degrees with a direction...,Xiaotangshan National Precision Agriculture Ex...,...,Cucumber,66.77 t ha−1,60572.74245,This value is given in tons per hectare.,2025-09-12 01:42:16,9,17.37 kg m−3,17.37,NaN,NaN
1,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Accumulated radiation combined with soil moist...,40.1786° N,40.178600,Original value was in degrees with a direction...,Xiaotangshan National Precision Agriculture Ex...,...,Cucumber,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,2025-09-12 01:42:16,9,20.52 kg m−3,20.52,NaN,NaN
2,NaN,pdf,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Crop evapotranspiration estimated method using...,40.1786° N,40.178600,Original value was in degrees with a direction...,Xiaotangshan National Precision Agriculture Ex...,...,Cucumber,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,2025-09-12 01:42:16,9,25.07 kg m−3,25.07,NaN,NaN
3,1550 m,pdf,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Soil water balance method,8°24′N,8.400000,Converted from degrees and minutes format (8°2...,"Melkassa Agricultural Research Center, Awash B...",...,Wheat,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,2025-09-12 04:01:02,9,1.42 kg m−3 (70% ETc),NaN,(70% ETc),False
4,542 m,pdf,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Regulated Deficit Irrigation,23º25′S,-23.416667,Converted from degrees and minutes format (23º...,"Maringá, Paraná, Brazil",...,Broccoli,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...",2025-09-12 04:06:41,9,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,521 m,pdf,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,"100% ETc, 4-day interval",34°20′N,34.333333,Original value was in degrees and minutes form...,"Yangling, Shaanxi, China",...,Tomato,"136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,2025-09-12 17:23:02,10,"32.7 kg/m3 (Spring 2023), 19.3 kg/m3 (Autumn 2...",19.30,Autumn 2023,True
312,NaN,pdf,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Timing of last irrigation,NaN,NaN,NaN,"Gezira Research Station Farm, Wad Medani, Sudan",...,Cotton,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,2025-09-12 23:58:14,8,0.45-0.71 kg/m3,0.71,Maximum value of the range 0.45-0.71 kg/m3,True
313,900 m,pdf,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,Film-Mulched Drip Irrigation,40°53′N,40.883333,Original value was in degrees and minutes form...,"Korla, Xinjiang, China",...,Cotton,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,2025-09-13 00:18:02,9,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",2.60,(W 0.8),True
314,900 m,pdf,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy150716

In [ ]:
combined_samples = next_df[["irrigation_scheduling_method", "plant_yield", "plant_yield_kg_per_hectare", "plant_yield_notes",
                  "location", "latitude", "latitude_decimal_degrees", "latitude_notes", "longitude", "longitude_decimal_degrees", "longitude_notes", "altitude", "plant_type", "plant_type_standardized", "plant_type_notes",
                  "water_productivity", "water_productivity_kg_per_m^3", "water_productivity_notes", "water_productivity_valid", "file_path", "doi", "doi_url", "processed_at", "content source", "relevance_score"]]
combined_samples

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,latitude_decimal_degrees,latitude_notes,longitude,longitude_decimal_degrees,...,water_productivity,water_productivity_kg_per_m^3,water_productivity_notes,water_productivity_valid,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,17.37 kg m−3,17.37,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,20.52 kg m−3,20.52,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,25.07 kg m−3,25.07,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,8.400000,Converted from degrees and minutes format (8°2...,39°21′E,39.350000,...,1.42 kg m−3 (70% ETc),NaN,(70% ETc),False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes format (23º...,51º57′W,-51.950000,...,NaN,NaN,NaN,NaN,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,"100% ETc, 4-day interval","136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,34.333333,Original value was in degrees and minutes form...,108°04′E,108.066667,...,"32.7 kg/m3 (Spring 2023), 19.3 kg/m3 (Autumn 2...",19.30,Autumn 2023,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10
312,Timing of last irrigation,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,NaN,NaN,NaN,NaN,...,0.45-0.71 kg/m3,0.71,Maximum value of the range 0.45-0.71 kg/m3,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8
313,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,86°56′E,86.933333,...,"2.53 kg m−3 (W 0.6), 2.60 kg m−3 (W 0.8), 1.54...",2.60,(W 0.8),True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9
314,Film-Mulched Drip Irrigation,"6218.07 

In [ ]:
combined_samples.to_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (6).csv")

Will standardize altitude after dropping / filling values.